## Imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
#from torchvision.datasets import ImageFolder
import torch.nn.functional as F
import torchvision.transforms as transforms
#rom torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torchvision.models import resnet18, ResNet18_Weights
# from efficientnet_pytorch import EfficientNet

import random
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import numpy as np

import torchvision
import matplotlib.pyplot as plt
import pandas as pd

from tqdm import tqdm
# from cv2 import INTER_CUBIC
import albumentations as A
from albumentations.pytorch import ToTensorV2

## Statistics

### Number of images

In [2]:
input_dir = 'folds-novos/'
img_filenames = set()
for filename in sorted(os.listdir(input_dir)):
    if filename.endswith('.txt'):
        with open(os.path.join(input_dir, filename)) as file:
            for line in file:
                if line.strip():
                    img_filenames.add(line.strip())

print(f'There are {len(img_filenames)} images')

There are 16824 images


### Number of unique patients

In [3]:
unique_pids = set()
for img_filename in img_filenames:
    unique_pids.add(img_filename.split('-')[3])
print(f'There are {len(unique_pids)} patients')

There are 13155 patients


## Porcentagem de duplicatas

In [4]:
n = 1 - len(unique_pids)/len(img_filenames)
print(f"There are {n*100:.2f} % same images")

There are 21.81 % same images


In [5]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        #self.backbone = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
        self.backbone = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.fc1 = nn.Sequential(nn.Linear(2*1000, 512),
                                nn.ReLU(inplace=True))

        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 2)

    def forward_once(self,x):
        output = self.backbone(x)
        output = output.view(output.size()[0], -1)
        return output
        
    def forward(self, x1, x2):
        output1 = self.forward_once(x1)
        output2 = self.forward_once(x2)

        output1 = output1.view(output1.size()[0], -1)
        output2 = output2.view(output2.size()[0], -1)
        
        output = torch.cat((output1, output2), 1)
        output = F.relu(self.fc1(output))
        output = F.relu(self.fc2(output))
        output = self.fc3(output)
        return output
        
        return output1, output2

In [6]:
#from efficientnet_pytorch import EfficientNet

#model = EfficientNet.from_pretrained('efficientnet-b0')
# model = resnet18(weights=ResNet18_Weights.DEFAULT)
siamese_net = SiameseNetwork()

In [7]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1): # TODO: threshold of the positive and negative results
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
 
    #valor 1 indica que as entradas pertencem à mesma classe
    #valor 0 indica que elas pertencem a classes diferentes
    
    def forward(self, output1, output2, label):
        euclidean_distance = torch.norm(output1 - output2, p=2, dim=1)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive

In [8]:
transform1 = A.Compose([
                A.Resize(height=224, width=224),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()])

In [9]:
class SiameseDataset(Dataset):
    def __init__(
        self,
        root_dir: str,
        fold_nums: list,
        fold_txt_dir: str,
        train,
        img_transforms
    ):
        
        super(SiameseDataset, self).__init__()
        
        self.root_dir = root_dir
        self.fold_nums = fold_nums
        self.img_transforms = img_transforms

        self.fold_txt_dir = fold_txt_dir
        self.train = train
        self.filepaths = []
        file, ids = [], []

        for i in fold_nums:
            filepath = os.path.join(root_dir, fold_txt_dir, f'fold-{i}.txt')
            with open(filepath) as txt_file:
                for line in txt_file:
                    if line.strip():
                        filename = line.strip()
                        if filename.startswith('panreport'):
                            self.filepaths.append(os.path.join(root_dir, filename))
                        else:
                            self.filepaths.append(os.path.join(root_dir, filename))

        # this maybe useful later for reproducibility
        self.filepaths.sort()
        # self.filepaths = self.filepaths[:1000]  #split do dataset para 1000 imagens
        print(f'Number of images is {len(self.filepaths)}')

    def __len__(self):
        return len(self.filepaths)

    def _get_pid(self, img_filepath):
        return img_filepath.split('/')[-1].split('-')[3]
        
    def __getitem__(self, idx):
        img_filepath_01 = self.filepaths[idx]
        target = random.randint(0, 1)

        # Nós escolhemos entre dois tipos de target:
        # 0 - Pacientes diferentes
        # 1 - Mesmo paciente
        # Se target == 1 e não encontramos um paciente correspondente a imagem
        # img_filepath_01, alteramos o target para 0 e selecionamos uma imagem aleatória
        
        if target == 1:
            random.shuffle(self.filepaths)
            for img_filepath_02 in self.filepaths:
                if self._get_pid(img_filepath_01) == self._get_pid(img_filepath_02) and img_filepath_01 != img_filepath_02:
                    break
            else:
                target = 0
                img_filepath_02 = random.choice(self.filepaths)
        else:
            random.shuffle(self.filepaths)
            for img_filepath_02 in self.filepaths:
                if self._get_pid(img_filepath_01) != self._get_pid(img_filepath_02):
                    break

        image1 = Image.open(img_filepath_01).convert('RGB')
        image2 = Image.open(img_filepath_02).convert('RGB')
        
        
        # apply transforms
        if self.train:
            image1, image2 = np.array(image1), np.array(image2)

            image1 = self.img_transforms(image = image1)['image']

            if img_filepath_01 == img_filepath_02: 
                image2 = self.img_transforms(image = image2)['image']
            else:
                image2 = self.img_transforms(image = image2)['image']
        else:
            image1, image2 = np.array(image1), np.array(image2)
            image1 = self.img_transforms(image = image1)['image']
            image2 = self.img_transforms(image = image2)['image']



        return (image1, image2, target, img_filepath_01, img_filepath_02)

In [10]:
# class siamese_dataset_2(Dataset):
#     def __init__(self,
#                 root_dir: str,
#                 fold_nums: list,
#                 fold_txt_dir: str,
#                 train,
#                 transforms1,
#                 transforms2):
        
#         super(siamese_dataset_2, self).__init__()
        
#         self.root_dir = root_dir
#         self.fold_nums = fold_nums
#         self.transforms1 = transforms1
#         self.transforms2 = transforms2

#         self.fold_txt_dir = fold_txt_dir
#         self.train = train
#         self.data = []
#         self.use_1st = False
#         self.dic = {}
#         file,ids = [],[]

#         for i in fold_nums:
#             filepath = os.path.join(root_dir, fold_txt_dir, f'fold-{i}.txt')
#             #filepath = os.path.join(root_dir, fold_txt_dir, f'{i:02d}.txt')
#             with open(filepath) as txt_file:
#                 for line in txt_file:
#                     img_relpath = line.strip()
#                     # if not self.use_1st and img_relpath.startswith('1st-set/'):
#                     #     continue
#                     filename = img_relpath.split('/')[-1]
#                     # pid = filename.split('-')[3]
#                     try:
#                         pid = filename.split('-')[3]
#                         # file.append(pid)
#                         # ids.append(pid)
#                         self.data.append(os.path.join(root_dir, img_relpath))
#                     except:
#                         print("An exception occurred")
                    

#         # this maybe useful later for reproducibility
#         self.data.sort()
#         df = pd.DataFrame(self.data, columns=['path'])
#         df['id'] = df.path.apply(lambda x : x.strip().split('/')[-1].split('-')[3])
        
#         for a, i in df.iterrows():
#             self.dic[i.id] = list(df[df.id == i.id].path) # cria um dicionario com id iguais


#     def __len__(self):
#         return len(self.data)
        
#     def __getitem__(self, idx):

        


#         target = np.random.randint(0,2) # 0 ou 1

#         image_1_id = random.choice(list(self.dic.keys())) # escolhe aleatoriamente a primeira imagem
#         images = self.dic[image_1_id]
#         image_1_filepath = random.choice(images)

#         if target == 1:
#             image_2_filepath = random.choice(images)

#         else: 
#         # Escolher um elemento aleatório da lista_total, se target igual a 0, image_1 nunca deve ser igual a image_2
#             image_2_id = image_1_id
#             while image_1_id == image_2_id:
#                 image_2_id = random.choice(list(self.dic.keys()))
#                 images = self.dic[image_2_id]
#                 image_2_filepath = random.choice(images)

#         image1 = Image.open(image_1_filepath).convert('RGB')
#         image2 = Image.open(image_2_filepath).convert('RGB')
        
        
#         # apply transforms
#         if self.train:
#             image1, image2 = np.array(image1), np.array(image2)

#             image1 = self.transforms1(image = image1)['image']

#             if image_1_filepath == image_2_filepath: 
#                 image2 = self.transforms2(image = image2)['image']
#             else:
#                 image2 = self.transforms1(image = image2)['image']
#         else:
#             image1, image2 = np.array(image1), np.array(image2)
#             image1 = self.transforms1(image = image1)['image']
#             image2 = self.transforms1(image = image2)['image']

#         return (image1, image2, target, image_1_filepath, image_2_filepath)

In [11]:
# definir diretório de imagens
data_dir = '/home/bernardo/github/sex-age-estimation/backup-bia/patch-1/data/pan-radiographs/images/'
#fold_txt_dir = '/home/bernardo/github/sex-age-estimation/backup-bia/patch-1/folds-filtrados/'
fold_txt_dir = '/home/bernardo/github/sex-age-estimation/backup-bia/patch-1/folds-novos/'

batch_size = 64
train_folds = [1,2,3,4]
val_fold = [5]
test_fold = [6] # fold-6 para teste

# folds 1,2,3,4,5 para treino e validação

# conjuntos de treinamento e teste

train_dataset = SiameseDataset(data_dir, train_folds, fold_txt_dir, True, transform1)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = SiameseDataset(data_dir, val_fold, fold_txt_dir, True, transform1)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

test_dataset = SiameseDataset(data_dir,test_fold, fold_txt_dir, False, transform1)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

Number of images is 11212
Number of images is 2808
Number of images is 2804


# visualizar imagens

## Checkpoint (pesos)

In [12]:
def load_checkpoint(checkpoint_path, siamese_net, optimizer):

    checkpoint = torch.load(checkpoint_path)

    # load variables
    siamese_net.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

    return step

def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("Saving checkpoint...")
    torch.save(state, filename)

## Metrics

In [13]:
def compute_accuracy(confusion):
    return (confusion['tp'] + confusion['tn'])/(confusion['tp'] + confusion['tn'] + confusion['fp'] + confusion['fn'])

def compute_precision(confusion):
    if confusion['tp'] + confusion['fp'] == 0:
        return 0
    return confusion['tp'] / (confusion['tp'] + confusion['fp'])

def compute_recall(confusion):
    if confusion['tp'] + confusion['fn'] == 0:
        return 0
    return confusion['tp'] / (confusion['tp'] + confusion['fn'])

def compute_f1(confusion):
    if 2 * confusion['tp'] + confusion['fp'] + confusion['fn'] == 0:
        return 0
    return 2 * confusion['tp'] / (2 * confusion['tp'] + confusion['fp'] + confusion['fn'])

## Primeiro treinamento

In [14]:
# definir função de perda e otimizador
# criterion = ContrastiveLoss()
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(siamese_net.parameters())
optimizer = torch.optim.SGD(siamese_net.parameters(), lr=0.01)

save_model = True
name = "ResNet-18"

# definir dispositivo de treinamento (CPU ou GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
siamese_net.to(device)

# Salvar os pesos e imprimir na tela o aumento das métricas
max_train_accuracy, max_train_f1, min_train_loss = 0, 0, 0

## Validation
max_val_accuracy, max_val_f1, min_val_loss = 0, 0, 0

epochs = 200
margin = 1  # ou qualquer outro valor desejado

# loop de treinamento
for epoch in range(1, epochs + 1):
    print(f'Epoch {epoch}/{epochs}')
    siamese_net.train()
    
    running_loss = 0.0
    confusion = {'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0}
    total = 0
    
    for img1, img2, label, x, y in tqdm(train_dataloader):
        img1, img2, label = img1.to(device), img2.to(device), label.to(device)

        optimizer.zero_grad()
        
        # output1, output2 = siamese_net(img1, img2)
        # loss = criterion(output1, output2, label)
        outputs = siamese_net(img1, img2)
        loss = criterion(outputs, label)
        
        # euclidean_distance = nn.functional.pairwise_distance(output1, output2)
        # predictions = (euclidean_distance >= margin)
        _, predictions = torch.max(outputs.data, 1)

        confusion['tp'] += ((predictions == 1) & (label == 1)).sum().item()
        confusion['tn'] += ((predictions == 0) & (label == 0)).sum().item()
        confusion['fp'] += ((predictions == 1) & (label == 0)).sum().item()
        confusion['fn'] += ((predictions == 0) & (label == 1)).sum().item()
        
        total += label.size(0)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        accuracy = compute_accuracy(confusion)
        precision = compute_precision(confusion)
        recall = compute_recall(confusion)
        f1 = compute_f1(confusion)
            
    print(f'Training loss: {running_loss / len(train_dataloader):.5f}')
    print(f'Train precision: {precision:.4f}')
    print(f'Train recall: {recall:.4f}')
    print(f'Train F1: {f1:.4f}')
    print(f'Training accuracy: {100*accuracy:.2f} (%)')
    print()

    if max_train_accuracy < accuracy:
        print(f'Accuracy aumentou de {max_train_accuracy:.4f}' + \
              f' para {accuracy:.4f} ({epoch}/{epochs})')

        max_train_accuracy = accuracy
        if save_model:
            checkpoint = {
                'state_dict': siamese_net.state_dict(),
                'optimizer': optimizer.state_dict()
            }
            save_checkpoint(checkpoint, filename=f'checkpoint-{name}-max-acc.pth.tar')
    
    if min_train_loss > loss:
        print(f'Loss de {min_train_loss:.2f}' + \
              f' para {loss:.2f} ({epoch}/{epochs})')

    if max_train_f1 < f1:
        print(f'F1 aumentou de {max_train_f1:.4f}' + \
              f' para {f1:.4f} ({epoch}/{epochs})')

        max_train_f1 = f1
        if save_model:
            checkpoint = {
                'state_dict': siamese_net.state_dict(),
                'optimizer': optimizer.state_dict()
            }
            save_checkpoint(checkpoint, filename=f'checkpoint-{name}-max-f1.pth.tar')

    print()

    with torch.no_grad():
    
        running_loss = 0.0 
        confusion = {'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0}
        total = 0
    
        for img1, img2, label, x, y in tqdm(val_dataloader):
            img1, img2, label = img1.to(device), img2.to(device), label.to(device)
    
            siamese_net.eval()
    
            # output1, output2 = siamese_net(img1, img2)
            # loss = criterion(output1, output2, label)
            outputs = siamese_net(img1, img2)
            loss = criterion(outputs, label)
            
            # euclidean_distance = nn.functional.pairwise_distance(output1, output2)
            # predictions = (euclidean_distance >= margin)
            _, predictions = torch.max(outputs.data, 1)

            confusion['tp'] += ((predictions == 1) & (label == 1)).sum().item()
            confusion['tn'] += ((predictions == 0) & (label == 0)).sum().item()
            confusion['fp'] += ((predictions == 1) & (label == 0)).sum().item()
            confusion['fn'] += ((predictions == 0) & (label == 1)).sum().item()
    
            total += label.size(0)
    
            # sum up the loss
            running_loss += loss.item()
    
            accuracy = compute_accuracy(confusion)
            precision = compute_precision(confusion)
            recall = compute_recall(confusion)
            f1 = compute_f1(confusion)

            val_loss = running_loss / len(val_dataloader)
    
        print(f'Validation loss: {running_loss / len(val_dataloader):.5f}')
        print(f'Validation precision: {precision:.4f}')
        print(f'Validation recall: {recall:.4f}')
        print(f'Validation F1: {f1:.4f}')
        print(f'Validation accuracy: {100*accuracy:.2f} (%)')
        print()
    
        if max_val_accuracy < accuracy:
            print(f'Accuracy aumentou de {max_val_accuracy:.4f}' + \
                  f' para {accuracy:.4f} ({epoch}/{epochs})')
    
            max_val_accuracy = accuracy
            if save_model:
                checkpoint = {
                    'state_dict': siamese_net.state_dict(),
                    'optimizer': optimizer.state_dict()
                }
                save_checkpoint(checkpoint, filename=f'checkpoint-{name}-max-acc.pth.tar')
    
        if max_val_f1 < f1:
            print(f'F1 aumentou de {max_val_f1:.4f}' + \
                  f' para {f1:.4f} ({epoch}/{epochs})')
    
            max_val_f1 = f1
            if save_model:
                checkpoint = {
                    'state_dict': siamese_net.state_dict(),
                    'optimizer': optimizer.state_dict()
                }
                save_checkpoint(checkpoint, filename=f'checkpoint-{name}-max-f1.pth.tar')
    
        if min_val_loss > loss:
            print(f'Loss de {min_val_loss:.2f}' + \
                  f' para {loss:.2f} ({epoch}/{epochs})')
    
        print()

Epoch 1/200


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:08<00:00,  5.16s/it]


Training loss: 0.38243
Train precision: 0.5927
Train recall: 0.2027
Train F1: 0.3021
Training accuracy: 81.82 (%)

Accuracy aumentou de 0.0000 para 0.8182 (1/200)
Saving checkpoint...
F1 aumentou de 0.0000 para 0.3021 (1/200)
Saving checkpoint...



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:23<00:00,  4.63s/it]


Validation loss: 0.34821
Validation precision: 0.7000
Validation recall: 0.3357
Validation F1: 0.4538
Validation accuracy: 83.80 (%)

Accuracy aumentou de 0.0000 para 0.8380 (1/200)
Saving checkpoint...
F1 aumentou de 0.0000 para 0.4538 (1/200)
Saving checkpoint...

Epoch 2/200


...

Validation loss: 0.34744
Validation precision: 0.9932
Validation recall: 0.8141
Validation F1: 0.8948
Validation accuracy: 96.33 (%)


Epoch 200/200


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:04<00:00,  5.48s/it]


Training loss: 0.00870
Train precision: 0.9922
Train recall: 0.9977
Train F1: 0.9949
Training accuracy: 99.80 (%)




100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:31<00:00,  4.82s/it]

Validation loss: 0.35956
Validation precision: 0.8861
Validation recall: 0.8586
Validation F1: 0.8722
Validation accuracy: 94.80 (%)




In [15]:
# Epoch 1/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:39<00:00,  5.00s/it]

# Training loss: 0.38793
# Train precision: 0.5968
# Train recall: 0.1579
# Train F1: 0.2497
# Training accuracy: 81.83 (%)

# Accuracy aumentou de 0.0000 para 0.8183 (1/200)
# Saving checkpoint...
# F1 aumentou de 0.0000 para 0.2497 (1/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:30<00:00,  4.79s/it]

# Validation loss: 0.34160
# Validation precision: 0.6821
# Validation recall: 0.2543
# Validation F1: 0.3705
# Validation accuracy: 83.90 (%)

# Accuracy aumentou de 0.0000 para 0.8390 (1/200)
# Saving checkpoint...
# F1 aumentou de 0.0000 para 0.3705 (1/200)
# Saving checkpoint...

# Epoch 2/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:56<00:00,  5.43s/it]

# Training loss: 0.30949
# Train precision: 0.6983
# Train recall: 0.5171
# Train F1: 0.5942
# Training accuracy: 86.21 (%)

# Accuracy aumentou de 0.8183 para 0.8621 (2/200)
# Saving checkpoint...
# F1 aumentou de 0.2497 para 0.5942 (2/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:31<00:00,  4.81s/it]

# Validation loss: 0.30742
# Validation precision: 0.7024
# Validation recall: 0.6733
# Validation F1: 0.6876
# Validation accuracy: 87.93 (%)

# Accuracy aumentou de 0.8390 para 0.8793 (2/200)
# Saving checkpoint...
# F1 aumentou de 0.3705 para 0.6876 (2/200)
# Saving checkpoint...

# Epoch 3/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:57<00:00,  5.44s/it]

# Training loss: 0.23619
# Train precision: 0.7690
# Train recall: 0.7049
# Train F1: 0.7356
# Training accuracy: 90.39 (%)

# Accuracy aumentou de 0.8621 para 0.9039 (3/200)
# Saving checkpoint...
# F1 aumentou de 0.5942 para 0.7356 (3/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:29<00:00,  4.76s/it]

# Validation loss: 0.26412
# Validation precision: 0.7578
# Validation recall: 0.6678
# Validation F1: 0.7100
# Validation accuracy: 88.71 (%)

# Accuracy aumentou de 0.8793 para 0.8871 (3/200)
# Saving checkpoint...
# F1 aumentou de 0.6876 para 0.7100 (3/200)
# Saving checkpoint...

# Epoch 4/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:49<00:00,  5.40s/it]

# Training loss: 0.18152
# Train precision: 0.8232
# Train recall: 0.7936
# Train F1: 0.8081
# Training accuracy: 92.61 (%)

# Accuracy aumentou de 0.9039 para 0.9261 (4/200)
# Saving checkpoint...
# F1 aumentou de 0.7356 para 0.8081 (4/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:28<00:00,  4.73s/it]

# Validation loss: 0.20394
# Validation precision: 0.8465
# Validation recall: 0.7225
# Validation F1: 0.7796
# Validation accuracy: 92.45 (%)

# Accuracy aumentou de 0.8871 para 0.9245 (4/200)
# Saving checkpoint...
# F1 aumentou de 0.7100 para 0.7796 (4/200)
# Saving checkpoint...

# Epoch 5/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:46<00:00,  5.38s/it]

# Training loss: 0.13870
# Train precision: 0.8710
# Train recall: 0.8595
# Train F1: 0.8652
# Training accuracy: 94.75 (%)

# Accuracy aumentou de 0.9261 para 0.9475 (5/200)
# Saving checkpoint...
# F1 aumentou de 0.8081 para 0.8652 (5/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:27<00:00,  4.73s/it]

# Validation loss: 0.15926
# Validation precision: 0.8586
# Validation recall: 0.8194
# Validation F1: 0.8385
# Validation accuracy: 94.09 (%)

# Accuracy aumentou de 0.9245 para 0.9409 (5/200)
# Saving checkpoint...
# F1 aumentou de 0.7796 para 0.8385 (5/200)
# Saving checkpoint...

# Epoch 6/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:43<00:00,  5.36s/it]

# Training loss: 0.11604
# Train precision: 0.8911
# Train recall: 0.8939
# Train F1: 0.8925
# Training accuracy: 95.75 (%)

# Accuracy aumentou de 0.9475 para 0.9575 (6/200)
# Saving checkpoint...
# F1 aumentou de 0.8652 para 0.8925 (6/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:28<00:00,  4.73s/it]

# Validation loss: 0.15527
# Validation precision: 0.8891
# Validation recall: 0.8394
# Validation F1: 0.8635
# Validation accuracy: 94.76 (%)

# Accuracy aumentou de 0.9409 para 0.9476 (6/200)
# Saving checkpoint...
# F1 aumentou de 0.8385 para 0.8635 (6/200)
# Saving checkpoint...

# Epoch 7/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:56<00:00,  5.43s/it]

# Training loss: 0.09552
# Train precision: 0.9048
# Train recall: 0.9150
# Train F1: 0.9099
# Training accuracy: 96.54 (%)

# Accuracy aumentou de 0.9575 para 0.9654 (7/200)
# Saving checkpoint...
# F1 aumentou de 0.8925 para 0.9099 (7/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:21<00:00,  4.57s/it]

# Validation loss: 0.19461
# Validation precision: 0.8998
# Validation recall: 0.7266
# Validation F1: 0.8040
# Validation accuracy: 92.98 (%)


# Epoch 8/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:30<00:00,  5.29s/it]

# Training loss: 0.07862
# Train precision: 0.9193
# Train recall: 0.9382
# Train F1: 0.9287
# Training accuracy: 97.19 (%)

# Accuracy aumentou de 0.9654 para 0.9719 (8/200)
# Saving checkpoint...
# F1 aumentou de 0.9099 para 0.9287 (8/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:20<00:00,  4.57s/it]

# Validation loss: 0.17255
# Validation precision: 0.9206
# Validation recall: 0.8043
# Validation F1: 0.8585
# Validation accuracy: 94.69 (%)


# Epoch 9/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:47<00:00,  5.04s/it]

# Training loss: 0.07041
# Train precision: 0.9272
# Train recall: 0.9426
# Train F1: 0.9349
# Training accuracy: 97.45 (%)

# Accuracy aumentou de 0.9719 para 0.9745 (9/200)
# Saving checkpoint...
# F1 aumentou de 0.9287 para 0.9349 (9/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:20<00:00,  4.56s/it]

# Validation loss: 0.13915
# Validation precision: 0.9089
# Validation recall: 0.8534
# Validation F1: 0.8803
# Validation accuracy: 95.26 (%)

# Accuracy aumentou de 0.9476 para 0.9526 (9/200)
# Saving checkpoint...
# F1 aumentou de 0.8635 para 0.8803 (9/200)
# Saving checkpoint...

# Epoch 10/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:20<00:00,  5.23s/it]

# Training loss: 0.05705
# Train precision: 0.9373
# Train recall: 0.9584
# Train F1: 0.9477
# Training accuracy: 97.98 (%)

# Accuracy aumentou de 0.9745 para 0.9798 (10/200)
# Saving checkpoint...
# F1 aumentou de 0.9349 para 0.9477 (10/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:21<00:00,  4.59s/it]

# Validation loss: 0.17463
# Validation precision: 0.9268
# Validation recall: 0.8559
# Validation F1: 0.8899
# Validation accuracy: 95.76 (%)

# Accuracy aumentou de 0.9526 para 0.9576 (10/200)
# Saving checkpoint...
# F1 aumentou de 0.8803 para 0.8899 (10/200)
# Saving checkpoint...

# Epoch 11/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:23<00:00,  5.25s/it]

# Training loss: 0.05878
# Train precision: 0.9334
# Train recall: 0.9584
# Train F1: 0.9457
# Training accuracy: 97.88 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:21<00:00,  4.58s/it]

# Validation loss: 0.16094
# Validation precision: 0.9174
# Validation recall: 0.8562
# Validation F1: 0.8857
# Validation accuracy: 95.41 (%)


# Epoch 12/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:23<00:00,  5.25s/it]

# Training loss: 0.05085
# Train precision: 0.9482
# Train recall: 0.9701
# Train F1: 0.9590
# Training accuracy: 98.37 (%)

# Accuracy aumentou de 0.9798 para 0.9837 (12/200)
# Saving checkpoint...
# F1 aumentou de 0.9477 para 0.9590 (12/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:21<00:00,  4.58s/it]

# Validation loss: 0.14486
# Validation precision: 0.8807
# Validation recall: 0.8759
# Validation F1: 0.8783
# Validation accuracy: 95.19 (%)


# Epoch 13/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:18<00:00,  5.22s/it]

# Training loss: 0.04909
# Train precision: 0.9484
# Train recall: 0.9701
# Train F1: 0.9591
# Training accuracy: 98.35 (%)

# F1 aumentou de 0.9590 para 0.9591 (13/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:19<00:00,  4.54s/it]

# Validation loss: 0.21487
# Validation precision: 0.8333
# Validation recall: 0.8752
# Validation F1: 0.8538
# Validation accuracy: 94.27 (%)


# Epoch 14/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:17<00:00,  5.22s/it]

# Training loss: 0.05003
# Train precision: 0.9414
# Train recall: 0.9735
# Train F1: 0.9571
# Training accuracy: 98.36 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:20<00:00,  4.55s/it]

# Validation loss: 0.25246
# Validation precision: 0.8893
# Validation recall: 0.8354
# Validation F1: 0.8615
# Validation accuracy: 94.48 (%)


# Epoch 15/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:19<00:00,  5.22s/it]

# Training loss: 0.04221
# Train precision: 0.9498
# Train recall: 0.9763
# Train F1: 0.9629
# Training accuracy: 98.56 (%)

# Accuracy aumentou de 0.9837 para 0.9856 (15/200)
# Saving checkpoint...
# F1 aumentou de 0.9591 para 0.9629 (15/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:20<00:00,  4.55s/it]

# Validation loss: 0.24234
# Validation precision: 0.8646
# Validation recall: 0.8603
# Validation F1: 0.8624
# Validation accuracy: 94.20 (%)


# Epoch 16/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:16<00:00,  5.21s/it]

# Training loss: 0.03626
# Train precision: 0.9581
# Train recall: 0.9782
# Train F1: 0.9680
# Training accuracy: 98.73 (%)

# Accuracy aumentou de 0.9856 para 0.9873 (16/200)
# Saving checkpoint...
# F1 aumentou de 0.9629 para 0.9680 (16/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:20<00:00,  4.56s/it]

# Validation loss: 0.14802
# Validation precision: 0.9308
# Validation recall: 0.8721
# Validation F1: 0.9005
# Validation accuracy: 96.40 (%)

# Accuracy aumentou de 0.9576 para 0.9640 (16/200)
# Saving checkpoint...
# F1 aumentou de 0.8899 para 0.9005 (16/200)
# Saving checkpoint...

# Epoch 17/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:19<00:00,  5.22s/it]

# Training loss: 0.03663
# Train precision: 0.9568
# Train recall: 0.9741
# Train F1: 0.9654
# Training accuracy: 98.65 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:27<00:00,  4.72s/it]

# Validation loss: 0.17408
# Validation precision: 0.8992
# Validation recall: 0.8457
# Validation F1: 0.8716
# Validation accuracy: 95.23 (%)


# Epoch 18/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:47<00:00,  5.38s/it]

# Training loss: 0.02917
# Train precision: 0.9644
# Train recall: 0.9834
# Train F1: 0.9738
# Training accuracy: 98.97 (%)

# Accuracy aumentou de 0.9873 para 0.9897 (18/200)
# Saving checkpoint...
# F1 aumentou de 0.9680 para 0.9738 (18/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:27<00:00,  4.72s/it]

# Validation loss: 0.17840
# Validation precision: 0.9576
# Validation recall: 0.8278
# Validation F1: 0.8880
# Validation accuracy: 95.94 (%)


# Epoch 19/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:44<00:00,  5.36s/it]

# Training loss: 0.03073
# Train precision: 0.9652
# Train recall: 0.9860
# Train F1: 0.9755
# Training accuracy: 99.02 (%)

# Accuracy aumentou de 0.9897 para 0.9902 (19/200)
# Saving checkpoint...
# F1 aumentou de 0.9738 para 0.9755 (19/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:26<00:00,  4.70s/it]

# Validation loss: 0.28301
# Validation precision: 0.8509
# Validation recall: 0.7712
# Validation F1: 0.8091
# Validation accuracy: 92.81 (%)


# Epoch 20/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:47<00:00,  5.39s/it]

# Training loss: 0.02849
# Train precision: 0.9675
# Train recall: 0.9851
# Train F1: 0.9762
# Training accuracy: 99.08 (%)

# Accuracy aumentou de 0.9902 para 0.9908 (20/200)
# Saving checkpoint...
# F1 aumentou de 0.9755 para 0.9762 (20/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:26<00:00,  4.70s/it]

# Validation loss: 0.26907
# Validation precision: 0.9324
# Validation recall: 0.8330
# Validation F1: 0.8799
# Validation accuracy: 95.44 (%)


# Epoch 21/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:50<00:00,  5.40s/it]

# Training loss: 0.02884
# Train precision: 0.9652
# Train recall: 0.9830
# Train F1: 0.9740
# Training accuracy: 99.01 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:28<00:00,  4.74s/it]

# Validation loss: 0.25507
# Validation precision: 0.9592
# Validation recall: 0.7055
# Validation F1: 0.8130
# Validation accuracy: 93.45 (%)


# Epoch 22/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:54<00:00,  5.42s/it]

# Training loss: 0.02997
# Train precision: 0.9688
# Train recall: 0.9837
# Train F1: 0.9762
# Training accuracy: 99.08 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:28<00:00,  4.73s/it]

# Validation loss: 0.27389
# Validation precision: 0.9598
# Validation recall: 0.7389
# Validation F1: 0.8350
# Validation accuracy: 94.62 (%)


# Epoch 23/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:46<00:00,  5.38s/it]

# Training loss: 0.02505
# Train precision: 0.9722
# Train recall: 0.9882
# Train F1: 0.9801
# Training accuracy: 99.22 (%)

# Accuracy aumentou de 0.9908 para 0.9922 (23/200)
# Saving checkpoint...
# F1 aumentou de 0.9762 para 0.9801 (23/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:28<00:00,  4.73s/it]

# Validation loss: 0.19451
# Validation precision: 0.9646
# Validation recall: 0.8180
# Validation F1: 0.8853
# Validation accuracy: 95.98 (%)


# Epoch 24/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:48<00:00,  5.39s/it]

# Training loss: 0.02408
# Train precision: 0.9701
# Train recall: 0.9903
# Train F1: 0.9801
# Training accuracy: 99.22 (%)

# Accuracy aumentou de 0.9922 para 0.9922 (24/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:28<00:00,  4.73s/it]

# Validation loss: 0.18568
# Validation precision: 0.9672
# Validation recall: 0.8262
# Validation F1: 0.8911
# Validation accuracy: 96.15 (%)


# Epoch 25/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:52<00:00,  5.41s/it]

# Training loss: 0.01833
# Train precision: 0.9790
# Train recall: 0.9923
# Train F1: 0.9856
# Training accuracy: 99.43 (%)

# Accuracy aumentou de 0.9922 para 0.9943 (25/200)
# Saving checkpoint...
# F1 aumentou de 0.9801 para 0.9856 (25/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:28<00:00,  4.74s/it]

# Validation loss: 0.20324
# Validation precision: 0.9821
# Validation recall: 0.8029
# Validation F1: 0.8835
# Validation accuracy: 95.87 (%)


# Epoch 26/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:44<00:00,  5.37s/it]

# Training loss: 0.02204
# Train precision: 0.9752
# Train recall: 0.9919
# Train F1: 0.9834
# Training accuracy: 99.34 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:27<00:00,  4.73s/it]

# Validation loss: 0.22543
# Validation precision: 0.9790
# Validation recall: 0.8265
# Validation F1: 0.8964
# Validation accuracy: 96.15 (%)


# Epoch 27/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:45<00:00,  5.37s/it]

# Training loss: 0.02035
# Train precision: 0.9791
# Train recall: 0.9887
# Train F1: 0.9839
# Training accuracy: 99.38 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:28<00:00,  4.75s/it]

# Validation loss: 0.29741
# Validation precision: 0.9620
# Validation recall: 0.8216
# Validation F1: 0.8863
# Validation accuracy: 95.83 (%)


# Epoch 28/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:50<00:00,  5.40s/it]

# Training loss: 0.01553
# Train precision: 0.9820
# Train recall: 0.9902
# Train F1: 0.9861
# Training accuracy: 99.46 (%)

# Accuracy aumentou de 0.9943 para 0.9946 (28/200)
# Saving checkpoint...
# F1 aumentou de 0.9856 para 0.9861 (28/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:27<00:00,  4.71s/it]

# Validation loss: 0.22640
# Validation precision: 0.9688
# Validation recall: 0.8112
# Validation F1: 0.8830
# Validation accuracy: 95.90 (%)


# Epoch 29/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:46<00:00,  5.38s/it]

# Training loss: 0.01924
# Train precision: 0.9780
# Train recall: 0.9893
# Train F1: 0.9836
# Training accuracy: 99.37 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:29<00:00,  4.76s/it]

# Validation loss: 0.22542
# Validation precision: 0.9553
# Validation recall: 0.8291
# Validation F1: 0.8877
# Validation accuracy: 96.15 (%)


# Epoch 30/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:44<00:00,  5.37s/it]

# Training loss: 0.02154
# Train precision: 0.9744
# Train recall: 0.9877
# Train F1: 0.9810
# Training accuracy: 99.25 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:28<00:00,  4.74s/it]

# Validation loss: 0.28466
# Validation precision: 0.9814
# Validation recall: 0.7932
# Validation F1: 0.8773
# Validation accuracy: 95.80 (%)


# Epoch 31/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:47<00:00,  5.38s/it]

# Training loss: 0.01992
# Train precision: 0.9753
# Train recall: 0.9924
# Train F1: 0.9838
# Training accuracy: 99.35 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:27<00:00,  4.72s/it]

# Validation loss: 0.20080
# Validation precision: 0.9632
# Validation recall: 0.8152
# Validation F1: 0.8830
# Validation accuracy: 96.05 (%)


# Epoch 32/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:48<00:00,  5.39s/it]

# Training loss: 0.01797
# Train precision: 0.9780
# Train recall: 0.9932
# Train F1: 0.9855
# Training accuracy: 99.43 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:28<00:00,  4.75s/it]

# Validation loss: 0.28089
# Validation precision: 0.9556
# Validation recall: 0.8083
# Validation F1: 0.8758
# Validation accuracy: 95.66 (%)


# Epoch 33/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:51<00:00,  5.41s/it]

# Training loss: 0.01463
# Train precision: 0.9808
# Train recall: 0.9915
# Train F1: 0.9861
# Training accuracy: 99.47 (%)

# Accuracy aumentou de 0.9946 para 0.9947 (33/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:29<00:00,  4.76s/it]

# Validation loss: 0.24387
# Validation precision: 0.9935
# Validation recall: 0.8241
# Validation F1: 0.9009
# Validation accuracy: 96.40 (%)

# F1 aumentou de 0.9005 para 0.9009 (33/200)
# Saving checkpoint...

# Epoch 34/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:46<00:00,  5.38s/it]

# Training loss: 0.01487
# Train precision: 0.9838
# Train recall: 0.9927
# Train F1: 0.9882
# Training accuracy: 99.54 (%)

# Accuracy aumentou de 0.9947 para 0.9954 (34/200)
# Saving checkpoint...
# F1 aumentou de 0.9861 para 0.9882 (34/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:29<00:00,  4.75s/it]

# Validation loss: 0.26692
# Validation precision: 0.9820
# Validation recall: 0.8048
# Validation F1: 0.8846
# Validation accuracy: 95.94 (%)


# Epoch 35/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:45<00:00,  5.37s/it]

# Training loss: 0.01449
# Train precision: 0.9836
# Train recall: 0.9922
# Train F1: 0.9879
# Training accuracy: 99.53 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:28<00:00,  4.73s/it]

# Validation loss: 0.24873
# Validation precision: 0.9806
# Validation recall: 0.7917
# Validation F1: 0.8761
# Validation accuracy: 95.41 (%)


# Epoch 36/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:48<00:00,  5.39s/it]

# Training loss: 0.01573
# Train precision: 0.9833
# Train recall: 0.9950
# Train F1: 0.9891
# Training accuracy: 99.57 (%)

# Accuracy aumentou de 0.9954 para 0.9957 (36/200)
# Saving checkpoint...
# F1 aumentou de 0.9882 para 0.9891 (36/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:29<00:00,  4.76s/it]

# Validation loss: 0.31677
# Validation precision: 0.9719
# Validation recall: 0.8032
# Validation F1: 0.8795
# Validation accuracy: 95.62 (%)


# Epoch 37/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:53<00:00,  5.42s/it]

# Training loss: 0.01765
# Train precision: 0.9781
# Train recall: 0.9873
# Train F1: 0.9827
# Training accuracy: 99.34 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:42<00:00,  5.07s/it]

# Validation loss: 0.21864
# Validation precision: 0.9745
# Validation recall: 0.8388
# Validation F1: 0.9016
# Validation accuracy: 96.44 (%)

# Accuracy aumentou de 0.9640 para 0.9644 (37/200)
# Saving checkpoint...
# F1 aumentou de 0.9009 para 0.9016 (37/200)
# Saving checkpoint...

# Epoch 38/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:13<00:00,  5.53s/it]

# Training loss: 0.01619
# Train precision: 0.9772
# Train recall: 0.9906
# Train F1: 0.9839
# Training accuracy: 99.38 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:33<00:00,  4.85s/it]

# Validation loss: 0.27572
# Validation precision: 0.9713
# Validation recall: 0.8103
# Validation F1: 0.8835
# Validation accuracy: 95.87 (%)


# Epoch 39/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:17<00:00,  5.55s/it]

# Training loss: 0.01495
# Train precision: 0.9855
# Train recall: 0.9941
# Train F1: 0.9898
# Training accuracy: 99.60 (%)

# Accuracy aumentou de 0.9957 para 0.9960 (39/200)
# Saving checkpoint...
# F1 aumentou de 0.9891 para 0.9898 (39/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:36<00:00,  4.93s/it]

# Validation loss: 0.40547
# Validation precision: 0.9523
# Validation recall: 0.6624
# Validation F1: 0.7813
# Validation accuracy: 92.84 (%)


# Epoch 40/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:24<00:00,  5.59s/it]

# Training loss: 0.01986
# Train precision: 0.9784
# Train recall: 0.9891
# Train F1: 0.9837
# Training accuracy: 99.36 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:36<00:00,  4.92s/it]

# Validation loss: 0.25319
# Validation precision: 0.8076
# Validation recall: 0.8993
# Validation F1: 0.8510
# Validation accuracy: 93.87 (%)


# Epoch 41/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:25<00:00,  5.60s/it]

# Training loss: 0.02015
# Train precision: 0.9811
# Train recall: 0.9888
# Train F1: 0.9850
# Training accuracy: 99.42 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:35<00:00,  4.89s/it]

# Validation loss: 0.25992
# Validation precision: 0.9333
# Validation recall: 0.8206
# Validation F1: 0.8733
# Validation accuracy: 95.23 (%)


# Epoch 42/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:32<00:00,  5.64s/it]

# Training loss: 0.02073
# Train precision: 0.9762
# Train recall: 0.9893
# Train F1: 0.9827
# Training accuracy: 99.33 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:45<00:00,  5.12s/it]

# Validation loss: 0.22134
# Validation precision: 0.9852
# Validation recall: 0.8369
# Validation F1: 0.9050
# Validation accuracy: 96.51 (%)

# Accuracy aumentou de 0.9644 para 0.9651 (42/200)
# Saving checkpoint...
# F1 aumentou de 0.9016 para 0.9050 (42/200)
# Saving checkpoint...

# Epoch 43/200


In [17]:
# Epoch 1/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:08<00:00,  5.16s/it]

# Training loss: 0.38243
# Train precision: 0.5927
# Train recall: 0.2027
# Train F1: 0.3021
# Training accuracy: 81.82 (%)

# Accuracy aumentou de 0.0000 para 0.8182 (1/200)
# Saving checkpoint...
# F1 aumentou de 0.0000 para 0.3021 (1/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:23<00:00,  4.63s/it]

# Validation loss: 0.34821
# Validation precision: 0.7000
# Validation recall: 0.3357
# Validation F1: 0.4538
# Validation accuracy: 83.80 (%)

# Accuracy aumentou de 0.0000 para 0.8380 (1/200)
# Saving checkpoint...
# F1 aumentou de 0.0000 para 0.4538 (1/200)
# Saving checkpoint...

# Epoch 2/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:37<00:00,  5.33s/it]

# Training loss: 0.29661
# Train precision: 0.7183
# Train recall: 0.5732
# Train F1: 0.6376
# Training accuracy: 87.17 (%)

# Accuracy aumentou de 0.8182 para 0.8717 (2/200)
# Saving checkpoint...
# F1 aumentou de 0.3021 para 0.6376 (2/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:22<00:00,  4.60s/it]

# Validation loss: 0.28547
# Validation precision: 0.7642
# Validation recall: 0.6978
# Validation F1: 0.7295
# Validation accuracy: 88.96 (%)

# Accuracy aumentou de 0.8380 para 0.8896 (2/200)
# Saving checkpoint...
# F1 aumentou de 0.4538 para 0.7295 (2/200)
# Saving checkpoint...

# Epoch 3/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:14<00:00,  4.51s/it]

# Training loss: 0.22138
# Train precision: 0.7899
# Train recall: 0.7700
# Train F1: 0.7798
# Training accuracy: 91.40 (%)

# Accuracy aumentou de 0.8717 para 0.9140 (3/200)
# Saving checkpoint...
# F1 aumentou de 0.6376 para 0.7798 (3/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:47<00:00,  3.82s/it]

# Validation loss: 0.22899
# Validation precision: 0.8197
# Validation recall: 0.6641
# Validation F1: 0.7338
# Validation accuracy: 90.95 (%)

# Accuracy aumentou de 0.8896 para 0.9095 (3/200)
# Saving checkpoint...
# F1 aumentou de 0.7295 para 0.7338 (3/200)
# Saving checkpoint...

# Epoch 4/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:24<00:00,  4.57s/it]

# Training loss: 0.16565
# Train precision: 0.8393
# Train recall: 0.8165
# Train F1: 0.8277
# Training accuracy: 93.51 (%)

# Accuracy aumentou de 0.9140 para 0.9351 (4/200)
# Saving checkpoint...
# F1 aumentou de 0.7798 para 0.8277 (4/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:50<00:00,  3.88s/it]

# Validation loss: 0.16421
# Validation precision: 0.8514
# Validation recall: 0.8416
# Validation F1: 0.8464
# Validation accuracy: 94.30 (%)

# Accuracy aumentou de 0.9095 para 0.9430 (4/200)
# Saving checkpoint...
# F1 aumentou de 0.7338 para 0.8464 (4/200)
# Saving checkpoint...

# Epoch 5/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:36<00:00,  4.64s/it]

# Training loss: 0.13228
# Train precision: 0.8766
# Train recall: 0.8766
# Train F1: 0.8766
# Training accuracy: 95.24 (%)

# Accuracy aumentou de 0.9351 para 0.9524 (5/200)
# Saving checkpoint...
# F1 aumentou de 0.8277 para 0.8766 (5/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.99s/it]

# Validation loss: 0.17428
# Validation precision: 0.8531
# Validation recall: 0.8046
# Validation F1: 0.8281
# Validation accuracy: 93.73 (%)


# Epoch 6/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:32<00:00,  4.62s/it]

# Training loss: 0.11737
# Train precision: 0.8818
# Train recall: 0.8996
# Train F1: 0.8906
# Training accuracy: 95.62 (%)

# Accuracy aumentou de 0.9524 para 0.9562 (6/200)
# Saving checkpoint...
# F1 aumentou de 0.8766 para 0.8906 (6/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.98s/it]

# Validation loss: 0.16949
# Validation precision: 0.9151
# Validation recall: 0.7307
# Validation F1: 0.8126
# Validation accuracy: 93.63 (%)


# Epoch 7/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:37<00:00,  4.64s/it]

# Training loss: 0.09294
# Train precision: 0.9103
# Train recall: 0.9237
# Train F1: 0.9169
# Training accuracy: 96.66 (%)

# Accuracy aumentou de 0.9562 para 0.9666 (7/200)
# Saving checkpoint...
# F1 aumentou de 0.8906 para 0.9169 (7/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  4.00s/it]

# Validation loss: 0.16541
# Validation precision: 0.8277
# Validation recall: 0.8829
# Validation F1: 0.8544
# Validation accuracy: 94.05 (%)

# F1 aumentou de 0.8464 para 0.8544 (7/200)
# Saving checkpoint...

# Epoch 8/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:43<00:00,  4.68s/it]

# Training loss: 0.08672
# Train precision: 0.9027
# Train recall: 0.9350
# Train F1: 0.9186
# Training accuracy: 96.86 (%)

# Accuracy aumentou de 0.9666 para 0.9686 (8/200)
# Saving checkpoint...
# F1 aumentou de 0.9169 para 0.9186 (8/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.98s/it]

# Validation loss: 0.13027
# Validation precision: 0.9246
# Validation recall: 0.8336
# Validation F1: 0.8768
# Validation accuracy: 95.33 (%)

# Accuracy aumentou de 0.9430 para 0.9533 (8/200)
# Saving checkpoint...
# F1 aumentou de 0.8544 para 0.8768 (8/200)
# Saving checkpoint...

# Epoch 9/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:37<00:00,  4.65s/it]

# Training loss: 0.07413
# Train precision: 0.9152
# Train recall: 0.9411
# Train F1: 0.9280
# Training accuracy: 97.28 (%)

# Accuracy aumentou de 0.9686 para 0.9728 (9/200)
# Saving checkpoint...
# F1 aumentou de 0.9186 para 0.9280 (9/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.97s/it]

# Validation loss: 0.19209
# Validation precision: 0.8769
# Validation recall: 0.8261
# Validation F1: 0.8507
# Validation accuracy: 94.30 (%)


# Epoch 10/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:36<00:00,  4.64s/it]

# Training loss: 0.06857
# Train precision: 0.9250
# Train recall: 0.9515
# Train F1: 0.9381
# Training accuracy: 97.57 (%)

# Accuracy aumentou de 0.9728 para 0.9757 (10/200)
# Saving checkpoint...
# F1 aumentou de 0.9280 para 0.9381 (10/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.16175
# Validation precision: 0.9269
# Validation recall: 0.8330
# Validation F1: 0.8775
# Validation accuracy: 95.58 (%)

# Accuracy aumentou de 0.9533 para 0.9558 (10/200)
# Saving checkpoint...
# F1 aumentou de 0.8768 para 0.8775 (10/200)
# Saving checkpoint...

# Epoch 11/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:41<00:00,  4.67s/it]

# Training loss: 0.06026
# Train precision: 0.9335
# Train recall: 0.9596
# Train F1: 0.9464
# Training accuracy: 97.91 (%)

# Accuracy aumentou de 0.9757 para 0.9791 (11/200)
# Saving checkpoint...
# F1 aumentou de 0.9381 para 0.9464 (11/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:53<00:00,  3.95s/it]

# Validation loss: 0.17927
# Validation precision: 0.8935
# Validation recall: 0.8030
# Validation F1: 0.8458
# Validation accuracy: 94.44 (%)


# Epoch 12/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:44<00:00,  5.03s/it]

# Training loss: 0.05948
# Train precision: 0.9373
# Train recall: 0.9586
# Train F1: 0.9479
# Training accuracy: 97.93 (%)

# Accuracy aumentou de 0.9791 para 0.9793 (12/200)
# Saving checkpoint...
# F1 aumentou de 0.9464 para 0.9479 (12/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.96s/it]

# Validation loss: 0.20212
# Validation precision: 0.9238
# Validation recall: 0.7491
# Validation F1: 0.8273
# Validation accuracy: 94.05 (%)


# Epoch 13/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:09<00:00,  4.83s/it]

# Training loss: 0.05142
# Train precision: 0.9404
# Train recall: 0.9612
# Train F1: 0.9507
# Training accuracy: 98.07 (%)

# Accuracy aumentou de 0.9793 para 0.9807 (13/200)
# Saving checkpoint...
# F1 aumentou de 0.9479 para 0.9507 (13/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:17<00:00,  4.50s/it]

# Validation loss: 0.17905
# Validation precision: 0.8360
# Validation recall: 0.8577
# Validation F1: 0.8467
# Validation accuracy: 94.02 (%)


# Epoch 14/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:03<00:00,  5.13s/it]

# Training loss: 0.05154
# Train precision: 0.9362
# Train recall: 0.9664
# Train F1: 0.9510
# Training accuracy: 98.05 (%)

# F1 aumentou de 0.9507 para 0.9510 (14/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:18<00:00,  4.51s/it]

# Validation loss: 0.17438
# Validation precision: 0.9556
# Validation recall: 0.8028
# Validation F1: 0.8726
# Validation accuracy: 95.30 (%)


# Epoch 15/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:06<00:00,  5.15s/it]

# Training loss: 0.04837
# Train precision: 0.9387
# Train recall: 0.9722
# Train F1: 0.9552
# Training accuracy: 98.27 (%)

# Accuracy aumentou de 0.9807 para 0.9827 (15/200)
# Saving checkpoint...
# F1 aumentou de 0.9510 para 0.9552 (15/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:18<00:00,  4.50s/it]

# Validation loss: 0.20606
# Validation precision: 0.9557
# Validation recall: 0.8319
# Validation F1: 0.8895
# Validation accuracy: 95.80 (%)

# Accuracy aumentou de 0.9558 para 0.9580 (15/200)
# Saving checkpoint...
# F1 aumentou de 0.8775 para 0.8895 (15/200)
# Saving checkpoint...

# Epoch 16/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:07<00:00,  5.15s/it]

# Training loss: 0.03993
# Train precision: 0.9513
# Train recall: 0.9770
# Train F1: 0.9640
# Training accuracy: 98.58 (%)

# Accuracy aumentou de 0.9827 para 0.9858 (16/200)
# Saving checkpoint...
# F1 aumentou de 0.9552 para 0.9640 (16/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:17<00:00,  4.49s/it]

# Validation loss: 0.18314
# Validation precision: 0.9302
# Validation recall: 0.8178
# Validation F1: 0.8704
# Validation accuracy: 95.62 (%)


# Epoch 17/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:06<00:00,  5.15s/it]

# Training loss: 0.04038
# Train precision: 0.9558
# Train recall: 0.9816
# Train F1: 0.9685
# Training accuracy: 98.80 (%)

# Accuracy aumentou de 0.9858 para 0.9880 (17/200)
# Saving checkpoint...
# F1 aumentou de 0.9640 para 0.9685 (17/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:18<00:00,  4.50s/it]

# Validation loss: 0.27357
# Validation precision: 0.8770
# Validation recall: 0.8095
# Validation F1: 0.8419
# Validation accuracy: 94.09 (%)


# Epoch 18/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:03<00:00,  5.13s/it]

# Training loss: 0.04613
# Train precision: 0.9482
# Train recall: 0.9705
# Train F1: 0.9592
# Training accuracy: 98.40 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:17<00:00,  4.49s/it]

# Validation loss: 0.23922
# Validation precision: 0.8745
# Validation recall: 0.8160
# Validation F1: 0.8442
# Validation accuracy: 94.23 (%)


# Epoch 19/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:05<00:00,  5.14s/it]

# Training loss: 0.04484
# Train precision: 0.9495
# Train recall: 0.9766
# Train F1: 0.9629
# Training accuracy: 98.56 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:17<00:00,  4.49s/it]

# Validation loss: 0.24425
# Validation precision: 0.9308
# Validation recall: 0.7215
# Validation F1: 0.8129
# Validation accuracy: 93.16 (%)


# Epoch 20/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:05<00:00,  5.14s/it]

# Training loss: 0.03240
# Train precision: 0.9599
# Train recall: 0.9848
# Train F1: 0.9722
# Training accuracy: 98.91 (%)

# Accuracy aumentou de 0.9880 para 0.9891 (20/200)
# Saving checkpoint...
# F1 aumentou de 0.9685 para 0.9722 (20/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.22470
# Validation precision: 0.9731
# Validation recall: 0.7806
# Validation F1: 0.8663
# Validation accuracy: 95.23 (%)


# Epoch 21/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:40<00:00,  4.66s/it]

# Training loss: 0.03398
# Train precision: 0.9573
# Train recall: 0.9811
# Train F1: 0.9691
# Training accuracy: 98.79 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.02s/it]

# Validation loss: 0.18702
# Validation precision: 0.9455
# Validation recall: 0.8283
# Validation F1: 0.8830
# Validation accuracy: 95.58 (%)


# Epoch 22/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:36<00:00,  4.64s/it]

# Training loss: 0.03224
# Train precision: 0.9625
# Train recall: 0.9802
# Train F1: 0.9713
# Training accuracy: 98.88 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:00<00:00,  4.11s/it]

# Validation loss: 0.20173
# Validation precision: 0.9354
# Validation recall: 0.8472
# Validation F1: 0.8891
# Validation accuracy: 96.01 (%)

# Accuracy aumentou de 0.9580 para 0.9601 (22/200)
# Saving checkpoint...

# Epoch 23/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:52<00:00,  5.07s/it]

# Training loss: 0.02943
# Train precision: 0.9672
# Train recall: 0.9873
# Train F1: 0.9771
# Training accuracy: 99.09 (%)

# Accuracy aumentou de 0.9891 para 0.9909 (23/200)
# Saving checkpoint...
# F1 aumentou de 0.9722 para 0.9771 (23/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:59<00:00,  4.07s/it]

# Validation loss: 0.19532
# Validation precision: 0.9468
# Validation recall: 0.8196
# Validation F1: 0.8786
# Validation accuracy: 95.80 (%)


# Epoch 24/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:44<00:00,  4.68s/it]

# Training loss: 0.02621
# Train precision: 0.9692
# Train recall: 0.9862
# Train F1: 0.9776
# Training accuracy: 99.13 (%)

# Accuracy aumentou de 0.9909 para 0.9913 (24/200)
# Saving checkpoint...
# F1 aumentou de 0.9771 para 0.9776 (24/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.98s/it]

# Validation loss: 0.24944
# Validation precision: 0.9290
# Validation recall: 0.8292
# Validation F1: 0.8763
# Validation accuracy: 95.66 (%)


# Epoch 25/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:40<00:00,  4.66s/it]

# Training loss: 0.02904
# Train precision: 0.9653
# Train recall: 0.9846
# Train F1: 0.9748
# Training accuracy: 99.00 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.98s/it]

# Validation loss: 0.21666
# Validation precision: 0.9556
# Validation recall: 0.8433
# Validation F1: 0.8959
# Validation accuracy: 96.26 (%)

# Accuracy aumentou de 0.9601 para 0.9626 (25/200)
# Saving checkpoint...
# F1 aumentou de 0.8895 para 0.8959 (25/200)
# Saving checkpoint...

# Epoch 26/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:39<00:00,  4.66s/it]

# Training loss: 0.02400
# Train precision: 0.9722
# Train recall: 0.9874
# Train F1: 0.9798
# Training accuracy: 99.19 (%)

# Accuracy aumentou de 0.9913 para 0.9919 (26/200)
# Saving checkpoint...
# F1 aumentou de 0.9776 para 0.9798 (26/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.99s/it]

# Validation loss: 0.21236
# Validation precision: 0.9543
# Validation recall: 0.7872
# Validation F1: 0.8627
# Validation accuracy: 95.26 (%)


# Epoch 27/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:40<00:00,  4.66s/it]

# Training loss: 0.02249
# Train precision: 0.9725
# Train recall: 0.9894
# Train F1: 0.9809
# Training accuracy: 99.25 (%)

# Accuracy aumentou de 0.9919 para 0.9925 (27/200)
# Saving checkpoint...
# F1 aumentou de 0.9798 para 0.9809 (27/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.18879
# Validation precision: 0.9495
# Validation recall: 0.8399
# Validation F1: 0.8913
# Validation accuracy: 96.08 (%)


# Epoch 28/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:32<00:00,  4.62s/it]

# Training loss: 0.02134
# Train precision: 0.9745
# Train recall: 0.9910
# Train F1: 0.9827
# Training accuracy: 99.30 (%)

# Accuracy aumentou de 0.9925 para 0.9930 (28/200)
# Saving checkpoint...
# F1 aumentou de 0.9809 para 0.9827 (28/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.23777
# Validation precision: 0.9506
# Validation recall: 0.7953
# Validation F1: 0.8661
# Validation accuracy: 95.12 (%)


# Epoch 29/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:42<00:00,  4.67s/it]

# Training loss: 0.02626
# Train precision: 0.9692
# Train recall: 0.9869
# Train F1: 0.9780
# Training accuracy: 99.15 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.96s/it]

# Validation loss: 0.21497
# Validation precision: 0.9872
# Validation recall: 0.8291
# Validation F1: 0.9013
# Validation accuracy: 96.40 (%)

# Accuracy aumentou de 0.9626 para 0.9640 (29/200)
# Saving checkpoint...
# F1 aumentou de 0.8959 para 0.9013 (29/200)
# Saving checkpoint...

# Epoch 30/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:36<00:00,  4.64s/it]

# Training loss: 0.02498
# Train precision: 0.9713
# Train recall: 0.9884
# Train F1: 0.9798
# Training accuracy: 99.22 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.03s/it]

# Validation loss: 0.33157
# Validation precision: 0.9612
# Validation recall: 0.7630
# Validation F1: 0.8507
# Validation accuracy: 95.05 (%)


# Epoch 31/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:40<00:00,  4.66s/it]

# Training loss: 0.02707
# Train precision: 0.9672
# Train recall: 0.9873
# Train F1: 0.9772
# Training accuracy: 99.09 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.99s/it]

# Validation loss: 0.27076
# Validation precision: 0.9652
# Validation recall: 0.8087
# Validation F1: 0.8801
# Validation accuracy: 95.69 (%)


# Epoch 32/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:35<00:00,  4.64s/it]

# Training loss: 0.02496
# Train precision: 0.9731
# Train recall: 0.9887
# Train F1: 0.9809
# Training accuracy: 99.27 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.99s/it]

# Validation loss: 0.21945
# Validation precision: 0.9524
# Validation recall: 0.8381
# Validation F1: 0.8916
# Validation accuracy: 96.19 (%)


# Epoch 33/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:37<00:00,  4.65s/it]

# Training loss: 0.02529
# Train precision: 0.9760
# Train recall: 0.9897
# Train F1: 0.9828
# Training accuracy: 99.34 (%)

# Accuracy aumentou de 0.9930 para 0.9934 (33/200)
# Saving checkpoint...
# F1 aumentou de 0.9827 para 0.9828 (33/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  4.00s/it]

# Validation loss: 0.19791
# Validation precision: 0.8770
# Validation recall: 0.8965
# Validation F1: 0.8867
# Validation accuracy: 95.58 (%)


# Epoch 34/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:38<00:00,  4.65s/it]

# Training loss: 0.03335
# Train precision: 0.9612
# Train recall: 0.9860
# Train F1: 0.9734
# Training accuracy: 98.97 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.03s/it]

# Validation loss: 0.21483
# Validation precision: 0.9152
# Validation recall: 0.8467
# Validation F1: 0.8796
# Validation accuracy: 95.58 (%)


# Epoch 35/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:38<00:00,  4.65s/it]

# Training loss: 0.02024
# Train precision: 0.9747
# Train recall: 0.9925
# Train F1: 0.9835
# Training accuracy: 99.37 (%)

# Accuracy aumentou de 0.9934 para 0.9937 (35/200)
# Saving checkpoint...
# F1 aumentou de 0.9828 para 0.9835 (35/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:59<00:00,  4.07s/it]

# Validation loss: 0.25423
# Validation precision: 0.9303
# Validation recall: 0.8537
# Validation F1: 0.8904
# Validation accuracy: 95.90 (%)


# Epoch 36/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:39<00:00,  4.66s/it]

# Training loss: 0.02190
# Train precision: 0.9752
# Train recall: 0.9911
# Train F1: 0.9831
# Training accuracy: 99.35 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:58<00:00,  4.05s/it]

# Validation loss: 0.28113
# Validation precision: 0.9717
# Validation recall: 0.7878
# Validation F1: 0.8701
# Validation accuracy: 95.62 (%)


# Epoch 37/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:52<00:00,  4.73s/it]

# Training loss: 0.01601
# Train precision: 0.9807
# Train recall: 0.9936
# Train F1: 0.9871
# Training accuracy: 99.49 (%)

# Accuracy aumentou de 0.9937 para 0.9949 (37/200)
# Saving checkpoint...
# F1 aumentou de 0.9835 para 0.9871 (37/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:58<00:00,  4.06s/it]

# Validation loss: 0.30752
# Validation precision: 0.9645
# Validation recall: 0.7565
# Validation F1: 0.8479
# Validation accuracy: 94.80 (%)


# Epoch 38/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:41<00:00,  4.67s/it]

# Training loss: 0.01972
# Train precision: 0.9790
# Train recall: 0.9901
# Train F1: 0.9845
# Training accuracy: 99.38 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.03s/it]

# Validation loss: 0.25120
# Validation precision: 0.9104
# Validation recall: 0.8249
# Validation F1: 0.8655
# Validation accuracy: 94.94 (%)


# Epoch 39/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:41<00:00,  4.67s/it]

# Training loss: 0.02554
# Train precision: 0.9705
# Train recall: 0.9859
# Train F1: 0.9782
# Training accuracy: 99.16 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.99s/it]

# Validation loss: 0.25958
# Validation precision: 0.9327
# Validation recall: 0.8072
# Validation F1: 0.8654
# Validation accuracy: 95.55 (%)


# Epoch 40/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:51<00:00,  4.72s/it]

# Training loss: 0.02222
# Train precision: 0.9726
# Train recall: 0.9895
# Train F1: 0.9810
# Training accuracy: 99.25 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.98s/it]

# Validation loss: 0.22605
# Validation precision: 0.9750
# Validation recall: 0.8110
# Validation F1: 0.8854
# Validation accuracy: 96.05 (%)


# Epoch 41/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:42<00:00,  4.67s/it]

# Training loss: 0.01967
# Train precision: 0.9785
# Train recall: 0.9903
# Train F1: 0.9844
# Training accuracy: 99.42 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.02s/it]

# Validation loss: 0.26590
# Validation precision: 0.9824
# Validation recall: 0.8022
# Validation F1: 0.8832
# Validation accuracy: 95.80 (%)


# Epoch 42/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:43<00:00,  4.68s/it]

# Training loss: 0.01947
# Train precision: 0.9793
# Train recall: 0.9916
# Train F1: 0.9854
# Training accuracy: 99.44 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.02s/it]

# Validation loss: 0.25053
# Validation precision: 0.9363
# Validation recall: 0.8476
# Validation F1: 0.8898
# Validation accuracy: 95.98 (%)


# Epoch 43/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:36<00:00,  4.64s/it]

# Training loss: 0.01692
# Train precision: 0.9796
# Train recall: 0.9924
# Train F1: 0.9860
# Training accuracy: 99.44 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.99s/it]

# Validation loss: 0.25422
# Validation precision: 0.9770
# Validation recall: 0.8555
# Validation F1: 0.9122
# Validation accuracy: 96.51 (%)

# Accuracy aumentou de 0.9640 para 0.9651 (43/200)
# Saving checkpoint...
# F1 aumentou de 0.9013 para 0.9122 (43/200)
# Saving checkpoint...

# Epoch 44/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:43<00:00,  4.68s/it]

# Training loss: 0.01632
# Train precision: 0.9809
# Train recall: 0.9924
# Train F1: 0.9866
# Training accuracy: 99.46 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.02s/it]

# Validation loss: 0.34819
# Validation precision: 0.9364
# Validation recall: 0.8324
# Validation F1: 0.8814
# Validation accuracy: 95.76 (%)


# Epoch 45/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:46<00:00,  4.69s/it]

# Training loss: 0.02542
# Train precision: 0.9719
# Train recall: 0.9909
# Train F1: 0.9813
# Training accuracy: 99.26 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  4.00s/it]

# Validation loss: 0.30463
# Validation precision: 0.9587
# Validation recall: 0.8271
# Validation F1: 0.8880
# Validation accuracy: 95.83 (%)


# Epoch 46/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:44<00:00,  4.69s/it]

# Training loss: 0.01935
# Train precision: 0.9761
# Train recall: 0.9902
# Train F1: 0.9831
# Training accuracy: 99.35 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.28499
# Validation precision: 0.9775
# Validation recall: 0.8253
# Validation F1: 0.8949
# Validation accuracy: 96.01 (%)


# Epoch 47/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:41<00:00,  4.67s/it]

# Training loss: 0.01584
# Train precision: 0.9815
# Train recall: 0.9922
# Train F1: 0.9868
# Training accuracy: 99.48 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.04s/it]

# Validation loss: 0.30718
# Validation precision: 0.9783
# Validation recall: 0.8039
# Validation F1: 0.8826
# Validation accuracy: 95.73 (%)


# Epoch 48/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:40<00:00,  4.66s/it]

# Training loss: 0.01555
# Train precision: 0.9835
# Train recall: 0.9949
# Train F1: 0.9892
# Training accuracy: 99.58 (%)

# Accuracy aumentou de 0.9949 para 0.9958 (48/200)
# Saving checkpoint...
# F1 aumentou de 0.9871 para 0.9892 (48/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.98s/it]

# Validation loss: 0.26141
# Validation precision: 0.9895
# Validation recall: 0.8163
# Validation F1: 0.8946
# Validation accuracy: 96.05 (%)


# Epoch 49/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:40<00:00,  4.66s/it]

# Training loss: 0.01740
# Train precision: 0.9780
# Train recall: 0.9918
# Train F1: 0.9849
# Training accuracy: 99.40 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.96s/it]

# Validation loss: 0.24057
# Validation precision: 0.9803
# Validation recall: 0.8217
# Validation F1: 0.8940
# Validation accuracy: 96.23 (%)


# Epoch 50/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:36<00:00,  4.64s/it]

# Training loss: 0.01484
# Train precision: 0.9840
# Train recall: 0.9928
# Train F1: 0.9884
# Training accuracy: 99.54 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.97s/it]

# Validation loss: 0.24889
# Validation precision: 0.9773
# Validation recall: 0.8086
# Validation F1: 0.8850
# Validation accuracy: 96.01 (%)


# Epoch 51/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:36<00:00,  4.64s/it]

# Training loss: 0.01044
# Train precision: 0.9876
# Train recall: 0.9960
# Train F1: 0.9917
# Training accuracy: 99.67 (%)

# Accuracy aumentou de 0.9958 para 0.9967 (51/200)
# Saving checkpoint...
# F1 aumentou de 0.9892 para 0.9917 (51/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:53<00:00,  3.95s/it]

# Validation loss: 0.26675
# Validation precision: 0.9688
# Validation recall: 0.8101
# Validation F1: 0.8824
# Validation accuracy: 95.87 (%)


# Epoch 52/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:38<00:00,  4.65s/it]

# Training loss: 0.01230
# Train precision: 0.9850
# Train recall: 0.9964
# Train F1: 0.9907
# Training accuracy: 99.63 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.99s/it]

# Validation loss: 0.47659
# Validation precision: 0.9890
# Validation recall: 0.6729
# Validation F1: 0.8009
# Validation accuracy: 93.63 (%)


# Epoch 53/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:37<00:00,  4.64s/it]

# Training loss: 0.01328
# Train precision: 0.9834
# Train recall: 0.9941
# Train F1: 0.9887
# Training accuracy: 99.55 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.99s/it]

# Validation loss: 0.27734
# Validation precision: 0.9815
# Validation recall: 0.8233
# Validation F1: 0.8955
# Validation accuracy: 96.47 (%)


# Epoch 54/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:32<00:00,  4.62s/it]

# Training loss: 0.01416
# Train precision: 0.9849
# Train recall: 0.9942
# Train F1: 0.9895
# Training accuracy: 99.58 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.33619
# Validation precision: 0.9428
# Validation recall: 0.8386
# Validation F1: 0.8877
# Validation accuracy: 95.69 (%)


# Epoch 55/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:53<00:00,  5.08s/it]

# Training loss: 0.01707
# Train precision: 0.9782
# Train recall: 0.9931
# Train F1: 0.9856
# Training accuracy: 99.44 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  4.00s/it]

# Validation loss: 0.21439
# Validation precision: 0.9768
# Validation recall: 0.8406
# Validation F1: 0.9036
# Validation accuracy: 96.47 (%)


# Epoch 56/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:45<00:00,  4.69s/it]

# Training loss: 0.01499
# Train precision: 0.9831
# Train recall: 0.9931
# Train F1: 0.9881
# Training accuracy: 99.54 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.26334
# Validation precision: 0.9778
# Validation recall: 0.8167
# Validation F1: 0.8900
# Validation accuracy: 96.12 (%)


# Epoch 57/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:44<00:00,  4.68s/it]

# Training loss: 0.01105
# Train precision: 0.9866
# Train recall: 0.9967
# Train F1: 0.9916
# Training accuracy: 99.68 (%)

# Accuracy aumentou de 0.9967 para 0.9968 (57/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.96s/it]

# Validation loss: 0.29964
# Validation precision: 0.9884
# Validation recall: 0.7821
# Validation F1: 0.8732
# Validation accuracy: 95.58 (%)


# Epoch 58/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:35<00:00,  4.63s/it]

# Training loss: 0.01389
# Train precision: 0.9856
# Train recall: 0.9944
# Train F1: 0.9900
# Training accuracy: 99.62 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:58<00:00,  4.05s/it]

# Validation loss: 0.31331
# Validation precision: 0.9827
# Validation recall: 0.7739
# Validation F1: 0.8659
# Validation accuracy: 95.62 (%)


# Epoch 59/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:40<00:00,  4.66s/it]

# Training loss: 0.01237
# Train precision: 0.9852
# Train recall: 0.9944
# Train F1: 0.9898
# Training accuracy: 99.61 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.04s/it]

# Validation loss: 0.28806
# Validation precision: 0.9806
# Validation recall: 0.8246
# Validation F1: 0.8959
# Validation accuracy: 96.23 (%)


# Epoch 60/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:46<00:00,  5.04s/it]

# Training loss: 0.01308
# Train precision: 0.9848
# Train recall: 0.9955
# Train F1: 0.9901
# Training accuracy: 99.61 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:18<00:00,  4.52s/it]

# Validation loss: 0.26423
# Validation precision: 0.9831
# Validation recall: 0.8115
# Validation F1: 0.8891
# Validation accuracy: 95.87 (%)


# Epoch 61/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:08<00:00,  5.16s/it]

# Training loss: 0.00996
# Train precision: 0.9880
# Train recall: 0.9973
# Train F1: 0.9926
# Training accuracy: 99.71 (%)

# Accuracy aumentou de 0.9968 para 0.9971 (61/200)
# Saving checkpoint...
# F1 aumentou de 0.9917 para 0.9926 (61/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:19<00:00,  4.52s/it]

# Validation loss: 0.24638
# Validation precision: 0.9773
# Validation recall: 0.8416
# Validation F1: 0.9044
# Validation accuracy: 96.44 (%)


# Epoch 62/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:10<00:00,  5.17s/it]

# Training loss: 0.01132
# Train precision: 0.9874
# Train recall: 0.9941
# Train F1: 0.9907
# Training accuracy: 99.63 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:20<00:00,  4.55s/it]

# Validation loss: 0.27257
# Validation precision: 0.9686
# Validation recall: 0.8591
# Validation F1: 0.9106
# Validation accuracy: 96.55 (%)

# Accuracy aumentou de 0.9651 para 0.9655 (62/200)
# Saving checkpoint...

# Epoch 63/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:09<00:00,  5.17s/it]

# Training loss: 0.01166
# Train precision: 0.9848
# Train recall: 0.9954
# Train F1: 0.9901
# Training accuracy: 99.62 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:17<00:00,  4.48s/it]

# Validation loss: 0.31745
# Validation precision: 0.9891
# Validation recall: 0.8300
# Validation F1: 0.9026
# Validation accuracy: 96.51 (%)


# Epoch 64/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:10<00:00,  5.17s/it]

# Training loss: 0.00991
# Train precision: 0.9889
# Train recall: 0.9949
# Train F1: 0.9919
# Training accuracy: 99.69 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:18<00:00,  4.52s/it]

# Validation loss: 0.25015
# Validation precision: 0.9887
# Validation recall: 0.8044
# Validation F1: 0.8871
# Validation accuracy: 96.05 (%)


# Epoch 65/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:06<00:00,  5.15s/it]

# Training loss: 0.00879
# Train precision: 0.9911
# Train recall: 0.9969
# Train F1: 0.9940
# Training accuracy: 99.76 (%)

# Accuracy aumentou de 0.9971 para 0.9976 (65/200)
# Saving checkpoint...
# F1 aumentou de 0.9926 para 0.9940 (65/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:19<00:00,  4.54s/it]

# Validation loss: 0.29102
# Validation precision: 0.9753
# Validation recall: 0.7905
# Validation F1: 0.8732
# Validation accuracy: 95.51 (%)


# Epoch 66/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:09<00:00,  5.17s/it]

# Training loss: 0.01124
# Train precision: 0.9889
# Train recall: 0.9916
# Train F1: 0.9902
# Training accuracy: 99.63 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:17<00:00,  4.49s/it]

# Validation loss: 0.31051
# Validation precision: 0.9099
# Validation recall: 0.8935
# Validation F1: 0.9016
# Validation accuracy: 96.15 (%)


# Epoch 67/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:09<00:00,  5.16s/it]

# Training loss: 0.01334
# Train precision: 0.9841
# Train recall: 0.9918
# Train F1: 0.9879
# Training accuracy: 99.53 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.28218
# Validation precision: 0.9706
# Validation recall: 0.8327
# Validation F1: 0.8964
# Validation accuracy: 96.19 (%)


# Epoch 68/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:41<00:00,  4.67s/it]

# Training loss: 0.01068
# Train precision: 0.9876
# Train recall: 0.9940
# Train F1: 0.9908
# Training accuracy: 99.64 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.99s/it]

# Validation loss: 0.38431
# Validation precision: 0.9776
# Validation recall: 0.7530
# Validation F1: 0.8507
# Validation accuracy: 94.55 (%)


# Epoch 69/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:40<00:00,  4.66s/it]

# Training loss: 0.01052
# Train precision: 0.9905
# Train recall: 0.9954
# Train F1: 0.9929
# Training accuracy: 99.72 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.29960
# Validation precision: 0.9595
# Validation recall: 0.8187
# Validation F1: 0.8835
# Validation accuracy: 95.55 (%)


# Epoch 70/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:41<00:00,  4.67s/it]

# Training loss: 0.01329
# Train precision: 0.9855
# Train recall: 0.9963
# Train F1: 0.9909
# Training accuracy: 99.64 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.02s/it]

# Validation loss: 0.33575
# Validation precision: 0.9931
# Validation recall: 0.8004
# Validation F1: 0.8864
# Validation accuracy: 96.05 (%)


# Epoch 71/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:39<00:00,  4.66s/it]

# Training loss: 0.00800
# Train precision: 0.9898
# Train recall: 0.9981
# Train F1: 0.9940
# Training accuracy: 99.77 (%)

# Accuracy aumentou de 0.9976 para 0.9977 (71/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.97s/it]

# Validation loss: 0.26670
# Validation precision: 0.9905
# Validation recall: 0.8038
# Validation F1: 0.8875
# Validation accuracy: 96.23 (%)


# Epoch 72/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:42<00:00,  4.67s/it]

# Training loss: 0.00829
# Train precision: 0.9903
# Train recall: 0.9967
# Train F1: 0.9935
# Training accuracy: 99.75 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.98s/it]

# Validation loss: 0.26008
# Validation precision: 0.9886
# Validation recall: 0.8093
# Validation F1: 0.8900
# Validation accuracy: 96.19 (%)


# Epoch 73/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:42<00:00,  4.67s/it]

# Training loss: 0.00753
# Train precision: 0.9900
# Train recall: 0.9963
# Train F1: 0.9932
# Training accuracy: 99.73 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.98s/it]

# Validation loss: 0.36093
# Validation precision: 0.9842
# Validation recall: 0.7982
# Validation F1: 0.8815
# Validation accuracy: 95.83 (%)


# Epoch 74/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:38<00:00,  4.65s/it]

# Training loss: 0.01031
# Train precision: 0.9870
# Train recall: 0.9972
# Train F1: 0.9921
# Training accuracy: 99.70 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.03s/it]

# Validation loss: 0.36017
# Validation precision: 0.9911
# Validation recall: 0.7799
# Validation F1: 0.8729
# Validation accuracy: 95.41 (%)


# Epoch 75/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:34<00:00,  4.63s/it]

# Training loss: 0.00732
# Train precision: 0.9902
# Train recall: 0.9981
# Train F1: 0.9941
# Training accuracy: 99.78 (%)

# Accuracy aumentou de 0.9977 para 0.9978 (75/200)
# Saving checkpoint...
# F1 aumentou de 0.9940 para 0.9941 (75/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  4.00s/it]

# Validation loss: 0.39510
# Validation precision: 0.9828
# Validation recall: 0.7854
# Validation F1: 0.8731
# Validation accuracy: 95.87 (%)


# Epoch 76/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:37<00:00,  4.64s/it]

# Training loss: 0.00877
# Train precision: 0.9909
# Train recall: 0.9950
# Train F1: 0.9929
# Training accuracy: 99.72 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.00s/it]

# Validation loss: 0.31549
# Validation precision: 0.9888
# Validation recall: 0.7907
# Validation F1: 0.8787
# Validation accuracy: 95.66 (%)


# Epoch 77/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:43<00:00,  4.68s/it]

# Training loss: 0.00815
# Train precision: 0.9897
# Train recall: 0.9972
# Train F1: 0.9934
# Training accuracy: 99.75 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.00s/it]

# Validation loss: 0.33015
# Validation precision: 0.9953
# Validation recall: 0.7922
# Validation F1: 0.8822
# Validation accuracy: 95.94 (%)


# Epoch 78/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:42<00:00,  4.67s/it]

# Training loss: 0.00704
# Train precision: 0.9922
# Train recall: 0.9977
# Train F1: 0.9950
# Training accuracy: 99.80 (%)

# Accuracy aumentou de 0.9978 para 0.9980 (78/200)
# Saving checkpoint...
# F1 aumentou de 0.9941 para 0.9950 (78/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.32164
# Validation precision: 0.9856
# Validation recall: 0.7969
# Validation F1: 0.8813
# Validation accuracy: 96.05 (%)


# Epoch 79/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:41<00:00,  4.67s/it]

# Training loss: 0.00819
# Train precision: 0.9885
# Train recall: 0.9944
# Train F1: 0.9915
# Training accuracy: 99.67 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.99s/it]

# Validation loss: 0.25848
# Validation precision: 0.9883
# Validation recall: 0.8103
# Validation F1: 0.8905
# Validation accuracy: 96.30 (%)


# Epoch 80/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:40<00:00,  4.66s/it]

# Training loss: 0.00807
# Train precision: 0.9907
# Train recall: 0.9967
# Train F1: 0.9937
# Training accuracy: 99.76 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  4.00s/it]

# Validation loss: 0.27049
# Validation precision: 0.9813
# Validation recall: 0.8325
# Validation F1: 0.9008
# Validation accuracy: 96.30 (%)


# Epoch 81/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:37<00:00,  4.65s/it]

# Training loss: 0.00664
# Train precision: 0.9907
# Train recall: 0.9977
# Train F1: 0.9942
# Training accuracy: 99.78 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.02s/it]

# Validation loss: 0.24838
# Validation precision: 0.9910
# Validation recall: 0.8209
# Validation F1: 0.8980
# Validation accuracy: 96.44 (%)


# Epoch 82/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:41<00:00,  4.67s/it]

# Training loss: 0.00768
# Train precision: 0.9888
# Train recall: 0.9962
# Train F1: 0.9925
# Training accuracy: 99.71 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.97s/it]

# Validation loss: 0.25412
# Validation precision: 0.9894
# Validation recall: 0.8158
# Validation F1: 0.8942
# Validation accuracy: 96.08 (%)


# Epoch 83/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:39<00:00,  4.66s/it]

# Training loss: 0.00799
# Train precision: 0.9901
# Train recall: 0.9967
# Train F1: 0.9934
# Training accuracy: 99.75 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  4.00s/it]

# Validation loss: 0.28826
# Validation precision: 0.9910
# Validation recall: 0.8308
# Validation F1: 0.9039
# Validation accuracy: 96.65 (%)

# Accuracy aumentou de 0.9655 para 0.9665 (83/200)
# Saving checkpoint...

# Epoch 84/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:39<00:00,  4.65s/it]

# Training loss: 0.01113
# Train precision: 0.9911
# Train recall: 0.9951
# Train F1: 0.9931
# Training accuracy: 99.72 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.98s/it]

# Validation loss: 0.28040
# Validation precision: 0.9607
# Validation recall: 0.8439
# Validation F1: 0.8986
# Validation accuracy: 96.26 (%)


# Epoch 85/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:40<00:00,  4.66s/it]

# Training loss: 0.01567
# Train precision: 0.9847
# Train recall: 0.9932
# Train F1: 0.9889
# Training accuracy: 99.56 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.97s/it]

# Validation loss: 0.27881
# Validation precision: 0.9780
# Validation recall: 0.8132
# Validation F1: 0.8880
# Validation accuracy: 96.01 (%)


# Epoch 86/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:38<00:00,  4.65s/it]

# Training loss: 0.01138
# Train precision: 0.9870
# Train recall: 0.9921
# Train F1: 0.9895
# Training accuracy: 99.60 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.00s/it]

# Validation loss: 0.26645
# Validation precision: 0.9175
# Validation recall: 0.8525
# Validation F1: 0.8838
# Validation accuracy: 95.83 (%)


# Epoch 87/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:41<00:00,  4.67s/it]

# Training loss: 0.01184
# Train precision: 0.9862
# Train recall: 0.9935
# Train F1: 0.9899
# Training accuracy: 99.61 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  4.00s/it]

# Validation loss: 0.26877
# Validation precision: 0.9667
# Validation recall: 0.7982
# Validation F1: 0.8744
# Validation accuracy: 95.55 (%)


# Epoch 88/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:43<00:00,  4.68s/it]

# Training loss: 0.01005
# Train precision: 0.9888
# Train recall: 0.9967
# Train F1: 0.9927
# Training accuracy: 99.72 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.31403
# Validation precision: 0.9847
# Validation recall: 0.8112
# Validation F1: 0.8895
# Validation accuracy: 96.01 (%)


# Epoch 89/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:37<00:00,  4.65s/it]

# Training loss: 0.01107
# Train precision: 0.9870
# Train recall: 0.9964
# Train F1: 0.9916
# Training accuracy: 99.67 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.04s/it]

# Validation loss: 0.33105
# Validation precision: 0.9948
# Validation recall: 0.7654
# Validation F1: 0.8652
# Validation accuracy: 95.73 (%)


# Epoch 90/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:43<00:00,  4.68s/it]

# Training loss: 0.00735
# Train precision: 0.9934
# Train recall: 0.9962
# Train F1: 0.9948
# Training accuracy: 99.80 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.43803
# Validation precision: 0.9821
# Validation recall: 0.7408
# Validation F1: 0.8445
# Validation accuracy: 94.98 (%)


# Epoch 91/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:40<00:00,  4.66s/it]

# Training loss: 0.00714
# Train precision: 0.9924
# Train recall: 0.9982
# Train F1: 0.9953
# Training accuracy: 99.81 (%)

# Accuracy aumentou de 0.9980 para 0.9981 (91/200)
# Saving checkpoint...
# F1 aumentou de 0.9950 para 0.9953 (91/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.98s/it]

# Validation loss: 0.36127
# Validation precision: 0.9772
# Validation recall: 0.8138
# Validation F1: 0.8881
# Validation accuracy: 95.76 (%)


# Epoch 92/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:42<00:00,  4.67s/it]

# Training loss: 0.00827
# Train precision: 0.9902
# Train recall: 0.9981
# Train F1: 0.9941
# Training accuracy: 99.78 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:59<00:00,  4.07s/it]

# Validation loss: 0.35099
# Validation precision: 0.9838
# Validation recall: 0.7755
# Validation F1: 0.8673
# Validation accuracy: 95.37 (%)


# Epoch 93/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:43<00:00,  4.68s/it]

# Training loss: 0.00892
# Train precision: 0.9898
# Train recall: 0.9958
# Train F1: 0.9928
# Training accuracy: 99.72 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.03s/it]

# Validation loss: 0.34335
# Validation precision: 0.9976
# Validation recall: 0.7910
# Validation F1: 0.8824
# Validation accuracy: 96.01 (%)


# Epoch 94/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:45<00:00,  4.69s/it]

# Training loss: 0.01174
# Train precision: 0.9900
# Train recall: 0.9968
# Train F1: 0.9934
# Training accuracy: 99.74 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.02s/it]

# Validation loss: 0.37181
# Validation precision: 0.9513
# Validation recall: 0.7109
# Validation F1: 0.8137
# Validation accuracy: 93.63 (%)


# Epoch 95/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:45<00:00,  4.69s/it]

# Training loss: 0.01165
# Train precision: 0.9878
# Train recall: 0.9941
# Train F1: 0.9909
# Training accuracy: 99.64 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.02s/it]

# Validation loss: 0.36890
# Validation precision: 0.9782
# Validation recall: 0.8032
# Validation F1: 0.8821
# Validation accuracy: 95.73 (%)


# Epoch 96/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:45<00:00,  4.69s/it]

# Training loss: 0.00891
# Train precision: 0.9915
# Train recall: 0.9962
# Train F1: 0.9939
# Training accuracy: 99.77 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:59<00:00,  4.09s/it]

# Validation loss: 0.49760
# Validation precision: 0.9630
# Validation recall: 0.7160
# Validation F1: 0.8213
# Validation accuracy: 93.55 (%)


# Epoch 97/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:44<00:00,  4.68s/it]

# Training loss: 0.01084
# Train precision: 0.9905
# Train recall: 0.9945
# Train F1: 0.9925
# Training accuracy: 99.71 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.27625
# Validation precision: 0.9886
# Validation recall: 0.8232
# Validation F1: 0.8983
# Validation accuracy: 96.51 (%)


# Epoch 98/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:36<00:00,  4.64s/it]

# Training loss: 0.00632
# Train precision: 0.9922
# Train recall: 0.9986
# Train F1: 0.9954
# Training accuracy: 99.82 (%)

# Accuracy aumentou de 0.9981 para 0.9982 (98/200)
# Saving checkpoint...
# F1 aumentou de 0.9953 para 0.9954 (98/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  4.00s/it]

# Validation loss: 0.32458
# Validation precision: 0.9845
# Validation recall: 0.7904
# Validation F1: 0.8768
# Validation accuracy: 95.55 (%)


# Epoch 99/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:38<00:00,  4.65s/it]

# Training loss: 0.00639
# Train precision: 0.9941
# Train recall: 0.9959
# Train F1: 0.9950
# Training accuracy: 99.80 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.97s/it]

# Validation loss: 0.30275
# Validation precision: 0.9822
# Validation recall: 0.8308
# Validation F1: 0.9002
# Validation accuracy: 96.51 (%)


# Epoch 100/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:37<00:00,  4.65s/it]

# Training loss: 0.00643
# Train precision: 0.9931
# Train recall: 0.9968
# Train F1: 0.9949
# Training accuracy: 99.80 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.02s/it]

# Validation loss: 0.43571
# Validation precision: 0.9956
# Validation recall: 0.7778
# Validation F1: 0.8733
# Validation accuracy: 95.30 (%)


# Epoch 101/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:39<00:00,  4.66s/it]

# Training loss: 0.00837
# Train precision: 0.9893
# Train recall: 0.9977
# Train F1: 0.9935
# Training accuracy: 99.75 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.02s/it]

# Validation loss: 0.29642
# Validation precision: 0.9915
# Validation recall: 0.8239
# Validation F1: 0.9000
# Validation accuracy: 96.30 (%)


# Epoch 102/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:40<00:00,  4.66s/it]

# Training loss: 0.00555
# Train precision: 0.9937
# Train recall: 0.9995
# Train F1: 0.9966
# Training accuracy: 99.87 (%)

# Accuracy aumentou de 0.9982 para 0.9987 (102/200)
# Saving checkpoint...
# F1 aumentou de 0.9954 para 0.9966 (102/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.96s/it]

# Validation loss: 0.31553
# Validation precision: 0.9870
# Validation recall: 0.8429
# Validation F1: 0.9093
# Validation accuracy: 96.76 (%)

# Accuracy aumentou de 0.9665 para 0.9676 (102/200)
# Saving checkpoint...

# Epoch 103/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:43<00:00,  4.68s/it]

# Training loss: 0.00696
# Train precision: 0.9922
# Train recall: 0.9968
# Train F1: 0.9945
# Training accuracy: 99.79 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.98s/it]

# Validation loss: 0.26718
# Validation precision: 0.9888
# Validation recall: 0.8185
# Validation F1: 0.8956
# Validation accuracy: 96.33 (%)


# Epoch 104/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:39<00:00,  4.65s/it]

# Training loss: 0.00975
# Train precision: 0.9922
# Train recall: 0.9963
# Train F1: 0.9942
# Training accuracy: 99.78 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.98s/it]

# Validation loss: 0.40969
# Validation precision: 0.9583
# Validation recall: 0.8410
# Validation F1: 0.8958
# Validation accuracy: 96.19 (%)


# Epoch 105/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:46<00:00,  4.70s/it]

# Training loss: 0.00877
# Train precision: 0.9877
# Train recall: 0.9954
# Train F1: 0.9916
# Training accuracy: 99.67 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:02<00:00,  4.14s/it]

# Validation loss: 0.35732
# Validation precision: 0.9906
# Validation recall: 0.7903
# Validation F1: 0.8792
# Validation accuracy: 95.87 (%)


# Epoch 106/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:04<00:00,  4.80s/it]

# Training loss: 0.00898
# Train precision: 0.9896
# Train recall: 0.9964
# Train F1: 0.9930
# Training accuracy: 99.72 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:01<00:00,  4.12s/it]

# Validation loss: 0.33436
# Validation precision: 0.9886
# Validation recall: 0.7883
# Validation F1: 0.8772
# Validation accuracy: 95.69 (%)


# Epoch 107/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:01<00:00,  4.78s/it]

# Training loss: 0.00637
# Train precision: 0.9909
# Train recall: 0.9991
# Train F1: 0.9950
# Training accuracy: 99.80 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:03<00:00,  4.18s/it]

# Validation loss: 0.34216
# Validation precision: 0.9860
# Validation recall: 0.7862
# Validation F1: 0.8749
# Validation accuracy: 95.69 (%)


# Epoch 108/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:02<00:00,  4.78s/it]

# Training loss: 0.00687
# Train precision: 0.9917
# Train recall: 0.9963
# Train F1: 0.9940
# Training accuracy: 99.77 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:02<00:00,  4.15s/it]

# Validation loss: 0.38044
# Validation precision: 0.9955
# Validation recall: 0.7768
# Validation F1: 0.8727
# Validation accuracy: 95.41 (%)


# Epoch 109/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:00<00:00,  4.78s/it]

# Training loss: 0.00693
# Train precision: 0.9925
# Train recall: 0.9976
# Train F1: 0.9951
# Training accuracy: 99.81 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:00<00:00,  4.10s/it]

# Validation loss: 0.34762
# Validation precision: 0.9321
# Validation recall: 0.8410
# Validation F1: 0.8842
# Validation accuracy: 95.90 (%)


# Epoch 110/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:01<00:00,  4.78s/it]

# Training loss: 0.01175
# Train precision: 0.9887
# Train recall: 0.9954
# Train F1: 0.9920
# Training accuracy: 99.69 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:00<00:00,  4.10s/it]

# Validation loss: 0.26540
# Validation precision: 0.9545
# Validation recall: 0.8540
# Validation F1: 0.9015
# Validation accuracy: 96.40 (%)


# Epoch 111/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:02<00:00,  4.79s/it]

# Training loss: 0.01362
# Train precision: 0.9876
# Train recall: 0.9945
# Train F1: 0.9910
# Training accuracy: 99.65 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:59<00:00,  4.07s/it]

# Validation loss: 0.36328
# Validation precision: 0.9158
# Validation recall: 0.8161
# Validation F1: 0.8631
# Validation accuracy: 94.84 (%)


# Epoch 112/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:59<00:00,  4.77s/it]

# Training loss: 0.00946
# Train precision: 0.9891
# Train recall: 0.9950
# Train F1: 0.9920
# Training accuracy: 99.69 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:00<00:00,  4.10s/it]

# Validation loss: 0.33026
# Validation precision: 0.9833
# Validation recall: 0.7672
# Validation F1: 0.8619
# Validation accuracy: 95.30 (%)


# Epoch 113/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:56<00:00,  4.75s/it]

# Training loss: 0.00398
# Train precision: 0.9954
# Train recall: 0.9977
# Train F1: 0.9966
# Training accuracy: 99.87 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:00<00:00,  4.09s/it]

# Validation loss: 0.31699
# Validation precision: 0.9779
# Validation recall: 0.8099
# Validation F1: 0.8860
# Validation accuracy: 95.94 (%)


# Epoch 114/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:44<00:00,  4.68s/it]

# Training loss: 0.00803
# Train precision: 0.9928
# Train recall: 0.9977
# Train F1: 0.9952
# Training accuracy: 99.81 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.97s/it]

# Validation loss: 0.32125
# Validation precision: 0.9889
# Validation recall: 0.8117
# Validation F1: 0.8916
# Validation accuracy: 96.15 (%)


# Epoch 115/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:45<00:00,  4.69s/it]

# Training loss: 0.00867
# Train precision: 0.9891
# Train recall: 0.9963
# Train F1: 0.9927
# Training accuracy: 99.71 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.44519
# Validation precision: 0.9874
# Validation recall: 0.7471
# Validation F1: 0.8506
# Validation accuracy: 95.09 (%)


# Epoch 116/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:48<00:00,  4.71s/it]

# Training loss: 0.00577
# Train precision: 0.9941
# Train recall: 0.9986
# Train F1: 0.9963
# Training accuracy: 99.86 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.29071
# Validation precision: 0.9879
# Validation recall: 0.8059
# Validation F1: 0.8877
# Validation accuracy: 96.33 (%)


# Epoch 117/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:44<00:00,  4.68s/it]

# Training loss: 0.00726
# Train precision: 0.9902
# Train recall: 0.9976
# Train F1: 0.9939
# Training accuracy: 99.77 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.26834
# Validation precision: 0.9951
# Validation recall: 0.7808
# Validation F1: 0.8750
# Validation accuracy: 95.87 (%)


# Epoch 118/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:45<00:00,  4.69s/it]

# Training loss: 0.00406
# Train precision: 0.9954
# Train recall: 0.9973
# Train F1: 0.9963
# Training accuracy: 99.86 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.03s/it]

# Validation loss: 0.35374
# Validation precision: 0.9932
# Validation recall: 0.7952
# Validation F1: 0.8832
# Validation accuracy: 95.90 (%)


# Epoch 119/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:44<00:00,  4.68s/it]

# Training loss: 0.00508
# Train precision: 0.9946
# Train recall: 0.9982
# Train F1: 0.9964
# Training accuracy: 99.86 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:58<00:00,  4.06s/it]

# Validation loss: 0.39210
# Validation precision: 0.9863
# Validation recall: 0.7989
# Validation F1: 0.8828
# Validation accuracy: 95.90 (%)


# Epoch 120/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:05<00:00,  4.80s/it]

# Training loss: 0.00677
# Train precision: 0.9934
# Train recall: 0.9972
# Train F1: 0.9953
# Training accuracy: 99.82 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:19<00:00,  4.54s/it]

# Validation loss: 0.36941
# Validation precision: 0.9609
# Validation recall: 0.8265
# Validation F1: 0.8887
# Validation accuracy: 95.83 (%)


# Epoch 121/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:12<00:00,  5.18s/it]

# Training loss: 0.00646
# Train precision: 0.9924
# Train recall: 0.9991
# Train F1: 0.9958
# Training accuracy: 99.83 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:19<00:00,  4.53s/it]

# Validation loss: 0.35301
# Validation precision: 0.9929
# Validation recall: 0.8027
# Validation F1: 0.8877
# Validation accuracy: 96.23 (%)


# Epoch 122/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:09<00:00,  5.17s/it]

# Training loss: 0.00414
# Train precision: 0.9941
# Train recall: 0.9991
# Train F1: 0.9966
# Training accuracy: 99.87 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:58<00:00,  4.07s/it]

# Validation loss: 0.27241
# Validation precision: 0.9933
# Validation recall: 0.8232
# Validation F1: 0.9003
# Validation accuracy: 96.47 (%)


# Epoch 123/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:41<00:00,  4.67s/it]

# Training loss: 0.00594
# Train precision: 0.9926
# Train recall: 0.9968
# Train F1: 0.9947
# Training accuracy: 99.79 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.03s/it]

# Validation loss: 0.30148
# Validation precision: 0.9774
# Validation recall: 0.8176
# Validation F1: 0.8903
# Validation accuracy: 95.83 (%)


# Epoch 124/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:47<00:00,  4.70s/it]

# Training loss: 0.00853
# Train precision: 0.9905
# Train recall: 0.9954
# Train F1: 0.9930
# Training accuracy: 99.72 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.00s/it]

# Validation loss: 0.26319
# Validation precision: 0.9842
# Validation recall: 0.8317
# Validation F1: 0.9016
# Validation accuracy: 96.62 (%)


# Epoch 125/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:43<00:00,  4.68s/it]

# Training loss: 0.00604
# Train precision: 0.9929
# Train recall: 0.9962
# Train F1: 0.9946
# Training accuracy: 99.79 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.99s/it]

# Validation loss: 0.31154
# Validation precision: 0.9929
# Validation recall: 0.7970
# Validation F1: 0.8842
# Validation accuracy: 96.08 (%)


# Epoch 126/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:16<00:00,  4.87s/it]

# Training loss: 0.00607
# Train precision: 0.9922
# Train recall: 0.9968
# Train F1: 0.9945
# Training accuracy: 99.79 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  4.00s/it]

# Validation loss: 0.44223
# Validation precision: 0.9930
# Validation recall: 0.7733
# Validation F1: 0.8695
# Validation accuracy: 95.48 (%)


# Epoch 127/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:44<00:00,  4.68s/it]

# Training loss: 0.00493
# Train precision: 0.9946
# Train recall: 0.9991
# Train F1: 0.9968
# Training accuracy: 99.88 (%)

# Accuracy aumentou de 0.9987 para 0.9988 (127/200)
# Saving checkpoint...
# F1 aumentou de 0.9966 para 0.9968 (127/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.38501
# Validation precision: 0.9934
# Validation recall: 0.8043
# Validation F1: 0.8889
# Validation accuracy: 95.98 (%)


# Epoch 128/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:44<00:00,  4.69s/it]

# Training loss: 0.00788
# Train precision: 0.9908
# Train recall: 0.9982
# Train F1: 0.9945
# Training accuracy: 99.79 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.40715
# Validation precision: 0.9903
# Validation recall: 0.7824
# Validation F1: 0.8742
# Validation accuracy: 95.80 (%)


# Epoch 129/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:39<00:00,  4.66s/it]

# Training loss: 0.00714
# Train precision: 0.9936
# Train recall: 0.9954
# Train F1: 0.9945
# Training accuracy: 99.79 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.98s/it]

# Validation loss: 0.43086
# Validation precision: 0.9901
# Validation recall: 0.7624
# Validation F1: 0.8614
# Validation accuracy: 95.41 (%)


# Epoch 130/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:44<00:00,  4.68s/it]

# Training loss: 0.00413
# Train precision: 0.9935
# Train recall: 0.9986
# Train F1: 0.9960
# Training accuracy: 99.85 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.99s/it]

# Validation loss: 0.32095
# Validation precision: 0.9840
# Validation recall: 0.8175
# Validation F1: 0.8930
# Validation accuracy: 96.33 (%)


# Epoch 131/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:43<00:00,  4.68s/it]

# Training loss: 0.00701
# Train precision: 0.9905
# Train recall: 0.9982
# Train F1: 0.9943
# Training accuracy: 99.78 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.36643
# Validation precision: 0.9886
# Validation recall: 0.7923
# Validation F1: 0.8797
# Validation accuracy: 95.76 (%)


# Epoch 132/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:49<00:00,  4.71s/it]

# Training loss: 0.00760
# Train precision: 0.9945
# Train recall: 0.9968
# Train F1: 0.9957
# Training accuracy: 99.83 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:58<00:00,  4.05s/it]

# Validation loss: 0.37080
# Validation precision: 0.9209
# Validation recall: 0.8056
# Validation F1: 0.8594
# Validation accuracy: 94.98 (%)


# Epoch 133/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [14:05<00:00,  4.80s/it]

# Training loss: 0.01027
# Train precision: 0.9881
# Train recall: 0.9945
# Train F1: 0.9913
# Training accuracy: 99.66 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.04s/it]

# Validation loss: 0.38269
# Validation precision: 0.9915
# Validation recall: 0.8171
# Validation F1: 0.8959
# Validation accuracy: 96.12 (%)


# Epoch 134/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:45<00:00,  4.69s/it]

# Training loss: 0.00498
# Train precision: 0.9944
# Train recall: 0.9995
# Train F1: 0.9969
# Training accuracy: 99.88 (%)

# Accuracy aumentou de 0.9988 para 0.9988 (134/200)
# Saving checkpoint...
# F1 aumentou de 0.9968 para 0.9969 (134/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.04s/it]

# Validation loss: 0.26940
# Validation precision: 0.9776
# Validation recall: 0.8138
# Validation F1: 0.8882
# Validation accuracy: 96.08 (%)


# Epoch 135/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:42<00:00,  4.68s/it]

# Training loss: 0.00526
# Train precision: 0.9935
# Train recall: 0.9977
# Train F1: 0.9956
# Training accuracy: 99.83 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.98s/it]

# Validation loss: 0.41252
# Validation precision: 0.9933
# Validation recall: 0.7872
# Validation F1: 0.8783
# Validation accuracy: 95.62 (%)


# Epoch 136/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:44<00:00,  4.68s/it]

# Training loss: 0.00611
# Train precision: 0.9919
# Train recall: 0.9971
# Train F1: 0.9945
# Training accuracy: 99.79 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.03s/it]

# Validation loss: 0.34942
# Validation precision: 0.9908
# Validation recall: 0.8026
# Validation F1: 0.8868
# Validation accuracy: 96.08 (%)


# Epoch 137/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:40<00:00,  4.66s/it]

# Training loss: 0.00709
# Train precision: 0.9931
# Train recall: 0.9972
# Train F1: 0.9951
# Training accuracy: 99.81 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.34831
# Validation precision: 0.9953
# Validation recall: 0.7978
# Validation F1: 0.8857
# Validation accuracy: 96.08 (%)


# Epoch 138/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:37<00:00,  4.65s/it]

# Training loss: 0.00485
# Train precision: 0.9946
# Train recall: 0.9982
# Train F1: 0.9964
# Training accuracy: 99.86 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  4.00s/it]

# Validation loss: 0.24989
# Validation precision: 0.9289
# Validation recall: 0.8772
# Validation F1: 0.9023
# Validation accuracy: 96.47 (%)


# Epoch 139/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:38<00:00,  4.65s/it]

# Training loss: 0.00709
# Train precision: 0.9908
# Train recall: 0.9981
# Train F1: 0.9945
# Training accuracy: 99.79 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:54<00:00,  3.98s/it]

# Validation loss: 0.30798
# Validation precision: 0.9933
# Validation recall: 0.8142
# Validation F1: 0.8949
# Validation accuracy: 96.26 (%)


# Epoch 140/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:36<00:00,  4.64s/it]

# Training loss: 0.00546
# Train precision: 0.9934
# Train recall: 0.9972
# Train F1: 0.9953
# Training accuracy: 99.82 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.23772
# Validation precision: 0.9484
# Validation recall: 0.8779
# Validation F1: 0.9118
# Validation accuracy: 96.58 (%)


# Epoch 141/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:37<00:00,  4.65s/it]

# Training loss: 0.00775
# Train precision: 0.9922
# Train recall: 0.9972
# Train F1: 0.9947
# Training accuracy: 99.79 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  4.00s/it]

# Validation loss: 0.34206
# Validation precision: 0.9217
# Validation recall: 0.8300
# Validation F1: 0.8735
# Validation accuracy: 95.26 (%)


# Epoch 142/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:41<00:00,  4.67s/it]

# Training loss: 0.00998
# Train precision: 0.9905
# Train recall: 0.9959
# Train F1: 0.9932
# Training accuracy: 99.73 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.40032
# Validation precision: 0.9938
# Validation recall: 0.8177
# Validation F1: 0.8972
# Validation accuracy: 96.08 (%)


# Epoch 143/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:39<00:00,  4.65s/it]

# Training loss: 0.00735
# Train precision: 0.9937
# Train recall: 0.9968
# Train F1: 0.9952
# Training accuracy: 99.81 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.02s/it]

# Validation loss: 0.33407
# Validation precision: 0.9789
# Validation recall: 0.8224
# Validation F1: 0.8938
# Validation accuracy: 96.08 (%)


# Epoch 144/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:40<00:00,  4.66s/it]

# Training loss: 0.00643
# Train precision: 0.9930
# Train recall: 0.9962
# Train F1: 0.9946
# Training accuracy: 99.79 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:57<00:00,  4.04s/it]

# Validation loss: 0.31967
# Validation precision: 0.9910
# Validation recall: 0.8212
# Validation F1: 0.8982
# Validation accuracy: 96.44 (%)


# Epoch 145/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:40<00:00,  4.66s/it]

# Training loss: 0.00553
# Train precision: 0.9931
# Train recall: 0.9974
# Train F1: 0.9952
# Training accuracy: 99.80 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.98s/it]

# Validation loss: 0.43934
# Validation precision: 0.9928
# Validation recall: 0.7615
# Validation F1: 0.8619
# Validation accuracy: 95.26 (%)


# Epoch 146/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:35<00:00,  4.63s/it]

# Training loss: 0.00476
# Train precision: 0.9946
# Train recall: 0.9986
# Train F1: 0.9966
# Training accuracy: 99.87 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.99s/it]

# Validation loss: 0.35944
# Validation precision: 0.9978
# Validation recall: 0.8161
# Validation F1: 0.8978
# Validation accuracy: 96.30 (%)


# Epoch 147/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:42<00:00,  4.67s/it]

# Training loss: 0.00300
# Train precision: 0.9972
# Train recall: 0.9986
# Train F1: 0.9979
# Training accuracy: 99.92 (%)

# Accuracy aumentou de 0.9988 para 0.9992 (147/200)
# Saving checkpoint...
# F1 aumentou de 0.9969 para 0.9979 (147/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.00s/it]

# Validation loss: 0.32657
# Validation precision: 0.9978
# Validation recall: 0.8102
# Validation F1: 0.8943
# Validation accuracy: 96.26 (%)


# Epoch 148/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:41<00:00,  4.67s/it]

# Training loss: 0.00455
# Train precision: 0.9945
# Train recall: 0.9995
# Train F1: 0.9970
# Training accuracy: 99.88 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  4.00s/it]

# Validation loss: 0.46081
# Validation precision: 0.9932
# Validation recall: 0.7861
# Validation F1: 0.8776
# Validation accuracy: 95.62 (%)


# Epoch 149/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:36<00:00,  4.64s/it]

# Training loss: 0.00523
# Train precision: 0.9936
# Train recall: 0.9991
# Train F1: 0.9963
# Training accuracy: 99.86 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:56<00:00,  4.01s/it]

# Validation loss: 0.35334
# Validation precision: 0.9952
# Validation recall: 0.7546
# Validation F1: 0.8583
# Validation accuracy: 95.16 (%)


# Epoch 150/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:25<00:00,  4.58s/it]

# Training loss: 0.00463
# Train precision: 0.9955
# Train recall: 0.9986
# Train F1: 0.9971
# Training accuracy: 99.88 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.98s/it]

# Validation loss: 0.39418
# Validation precision: 0.9898
# Validation recall: 0.8083
# Validation F1: 0.8899
# Validation accuracy: 95.73 (%)


# Epoch 151/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:38<00:00,  4.65s/it]

# Training loss: 0.00649
# Train precision: 0.9960
# Train recall: 0.9991
# Train F1: 0.9976
# Training accuracy: 99.90 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:55<00:00,  3.99s/it]

# Validation loss: 0.46411
# Validation precision: 0.9378
# Validation recall: 0.6642
# Validation F1: 0.7777
# Validation accuracy: 92.63 (%)


# Epoch 152/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:39<00:00,  4.65s/it]

# Training loss: 0.00638
# Train precision: 0.9934
# Train recall: 0.9971
# Train F1: 0.9953
# Training accuracy: 99.82 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:58<00:00,  4.05s/it]

# Validation loss: 0.39579
# Validation precision: 0.9957
# Validation recall: 0.8226
# Validation F1: 0.9009
# Validation accuracy: 96.40 (%)


# Epoch 153/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:47<00:00,  4.70s/it]

# Training loss: 0.00418
# Train precision: 0.9945
# Train recall: 0.9977
# Train F1: 0.9961
# Training accuracy: 99.85 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:58<00:00,  4.05s/it]

# Validation loss: 0.34429
# Validation precision: 0.9905
# Validation recall: 0.7883
# Validation F1: 0.8779
# Validation accuracy: 95.87 (%)


# Epoch 154/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:48<00:00,  4.70s/it]

# Training loss: 0.00544
# Train precision: 0.9939
# Train recall: 0.9963
# Train F1: 0.9951
# Training accuracy: 99.81 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [02:59<00:00,  4.08s/it]

# Validation loss: 0.45746
# Validation precision: 0.9868
# Validation recall: 0.7795
# Validation F1: 0.8710
# Validation accuracy: 95.26 (%)


# Epoch 155/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [13:42<00:00,  4.67s/it]

# Training loss: 0.00579
# Train precision: 0.9928
# Train recall: 0.9982
# Train F1: 0.9955
# Training accuracy: 99.82 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:31<00:00,  4.80s/it]

# Validation loss: 0.45823
# Validation precision: 0.9908
# Validation recall: 0.7784
# Validation F1: 0.8718
# Validation accuracy: 95.48 (%)


# Epoch 156/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:00<00:00,  5.46s/it]

# Training loss: 0.00477
# Train precision: 0.9953
# Train recall: 0.9976
# Train F1: 0.9965
# Training accuracy: 99.87 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:31<00:00,  4.82s/it]

# Validation loss: 0.48880
# Validation precision: 0.9949
# Validation recall: 0.7247
# Validation F1: 0.8386
# Validation accuracy: 94.69 (%)


# Epoch 157/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:03<00:00,  5.47s/it]

# Training loss: 0.00474
# Train precision: 0.9934
# Train recall: 0.9981
# Train F1: 0.9958
# Training accuracy: 99.84 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:31<00:00,  4.81s/it]

# Validation loss: 0.36296
# Validation precision: 0.9864
# Validation recall: 0.8089
# Validation F1: 0.8889
# Validation accuracy: 96.12 (%)


# Epoch 158/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:01<00:00,  5.47s/it]

# Training loss: 0.00420
# Train precision: 0.9953
# Train recall: 0.9972
# Train F1: 0.9962
# Training accuracy: 99.86 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:30<00:00,  4.79s/it]

# Validation loss: 0.35991
# Validation precision: 0.9936
# Validation recall: 0.7969
# Validation F1: 0.8845
# Validation accuracy: 95.66 (%)


# Epoch 159/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:02<00:00,  5.47s/it]

# Training loss: 0.00463
# Train precision: 0.9944
# Train recall: 0.9986
# Train F1: 0.9965
# Training accuracy: 99.87 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.84s/it]

# Validation loss: 0.51568
# Validation precision: 0.9949
# Validation recall: 0.6909
# Validation F1: 0.8155
# Validation accuracy: 93.73 (%)


# Epoch 160/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:06<00:00,  5.49s/it]

# Training loss: 0.00302
# Train precision: 0.9968
# Train recall: 0.9991
# Train F1: 0.9979
# Training accuracy: 99.92 (%)

# F1 aumentou de 0.9979 para 0.9979 (160/200)
# Saving checkpoint...

# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:31<00:00,  4.81s/it]

# Validation loss: 0.40896
# Validation precision: 0.9954
# Validation recall: 0.8056
# Validation F1: 0.8905
# Validation accuracy: 96.19 (%)


# Epoch 161/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:04<00:00,  5.48s/it]

# Training loss: 0.00492
# Train precision: 0.9950
# Train recall: 0.9986
# Train F1: 0.9968
# Training accuracy: 99.88 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.84s/it]

# Validation loss: 0.32731
# Validation precision: 0.9852
# Validation recall: 0.8488
# Validation F1: 0.9119
# Validation accuracy: 96.79 (%)

# Accuracy aumentou de 0.9676 para 0.9679 (161/200)
# Saving checkpoint...

# Epoch 162/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:59<00:00,  5.45s/it]

# Training loss: 0.00489
# Train precision: 0.9926
# Train recall: 0.9986
# Train F1: 0.9956
# Training accuracy: 99.83 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:31<00:00,  4.82s/it]

# Validation loss: 0.47958
# Validation precision: 0.9926
# Validation recall: 0.7795
# Validation F1: 0.8732
# Validation accuracy: 95.83 (%)


# Epoch 163/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:02<00:00,  5.47s/it]

# Training loss: 0.00409
# Train precision: 0.9959
# Train recall: 0.9982
# Train F1: 0.9970
# Training accuracy: 99.88 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:30<00:00,  4.78s/it]

# Validation loss: 0.49518
# Validation precision: 0.9927
# Validation recall: 0.7312
# Validation F1: 0.8421
# Validation accuracy: 94.55 (%)


# Epoch 164/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:00<00:00,  5.46s/it]

# Training loss: 0.00451
# Train precision: 0.9929
# Train recall: 0.9981
# Train F1: 0.9955
# Training accuracy: 99.83 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:31<00:00,  4.80s/it]

# Validation loss: 0.40420
# Validation precision: 0.9934
# Validation recall: 0.7933
# Validation F1: 0.8822
# Validation accuracy: 95.69 (%)


# Epoch 165/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:05<00:00,  5.49s/it]

# Training loss: 0.00311
# Train precision: 0.9968
# Train recall: 0.9986
# Train F1: 0.9977
# Training accuracy: 99.91 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:30<00:00,  4.79s/it]

# Validation loss: 0.33626
# Validation precision: 0.9917
# Validation recall: 0.8463
# Validation F1: 0.9133
# Validation accuracy: 96.76 (%)

# F1 aumentou de 0.9122 para 0.9133 (165/200)
# Saving checkpoint...

# Epoch 166/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:03<00:00,  5.47s/it]

# Training loss: 0.00885
# Train precision: 0.9958
# Train recall: 0.9972
# Train F1: 0.9965
# Training accuracy: 99.87 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.82s/it]

# Validation loss: 0.47231
# Validation precision: 0.8346
# Validation recall: 0.8393
# Validation F1: 0.8369
# Validation accuracy: 93.77 (%)


# Epoch 167/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:02<00:00,  5.47s/it]

# Training loss: 0.01616
# Train precision: 0.9830
# Train recall: 0.9939
# Train F1: 0.9884
# Training accuracy: 99.55 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:33<00:00,  4.85s/it]

# Validation loss: 0.31732
# Validation precision: 0.9925
# Validation recall: 0.7763
# Validation F1: 0.8712
# Validation accuracy: 95.80 (%)


# Epoch 168/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:00<00:00,  5.46s/it]

# Training loss: 0.00490
# Train precision: 0.9946
# Train recall: 0.9968
# Train F1: 0.9957
# Training accuracy: 99.83 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:33<00:00,  4.85s/it]

# Validation loss: 0.33347
# Validation precision: 0.9878
# Validation recall: 0.7849
# Validation F1: 0.8747
# Validation accuracy: 95.87 (%)


# Epoch 169/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:01<00:00,  5.46s/it]

# Training loss: 0.00730
# Train precision: 0.9908
# Train recall: 0.9951
# Train F1: 0.9930
# Training accuracy: 99.74 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:31<00:00,  4.81s/it]

# Validation loss: 0.36681
# Validation precision: 0.9955
# Validation recall: 0.7932
# Validation F1: 0.8829
# Validation accuracy: 95.83 (%)


# Epoch 170/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:00<00:00,  5.46s/it]

# Training loss: 0.00364
# Train precision: 0.9968
# Train recall: 0.9986
# Train F1: 0.9977
# Training accuracy: 99.91 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.83s/it]

# Validation loss: 0.43452
# Validation precision: 0.9952
# Validation recall: 0.7665
# Validation F1: 0.8660
# Validation accuracy: 95.41 (%)


# Epoch 171/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:01<00:00,  5.47s/it]

# Training loss: 0.00539
# Train precision: 0.9945
# Train recall: 0.9982
# Train F1: 0.9963
# Training accuracy: 99.86 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.84s/it]

# Validation loss: 0.52340
# Validation precision: 0.9878
# Validation recall: 0.7486
# Validation F1: 0.8517
# Validation accuracy: 94.98 (%)


# Epoch 172/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:02<00:00,  5.47s/it]

# Training loss: 0.00676
# Train precision: 0.9931
# Train recall: 0.9977
# Train F1: 0.9954
# Training accuracy: 99.82 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:33<00:00,  4.84s/it]

# Validation loss: 0.39666
# Validation precision: 0.9953
# Validation recall: 0.7790
# Validation F1: 0.8740
# Validation accuracy: 95.66 (%)


# Epoch 173/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:56<00:00,  5.44s/it]

# Training loss: 0.00844
# Train precision: 0.9939
# Train recall: 0.9981
# Train F1: 0.9960
# Training accuracy: 99.85 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.84s/it]

# Validation loss: 0.54820
# Validation precision: 0.7512
# Validation recall: 0.8415
# Validation F1: 0.7938
# Validation accuracy: 91.45 (%)


# Epoch 174/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:03<00:00,  5.47s/it]

# Training loss: 0.01719
# Train precision: 0.9836
# Train recall: 0.9913
# Train F1: 0.9874
# Training accuracy: 99.51 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.84s/it]

# Validation loss: 0.47187
# Validation precision: 0.9933
# Validation recall: 0.7907
# Validation F1: 0.8805
# Validation accuracy: 95.73 (%)


# Epoch 175/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:00<00:00,  5.46s/it]

# Training loss: 0.00839
# Train precision: 0.9885
# Train recall: 0.9958
# Train F1: 0.9922
# Training accuracy: 99.70 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:33<00:00,  4.84s/it]

# Validation loss: 0.24496
# Validation precision: 0.9834
# Validation recall: 0.8716
# Validation F1: 0.9241
# Validation accuracy: 97.22 (%)

# Accuracy aumentou de 0.9679 para 0.9722 (175/200)
# Saving checkpoint...
# F1 aumentou de 0.9133 para 0.9241 (175/200)
# Saving checkpoint...

# Epoch 176/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:01<00:00,  5.46s/it]

# Training loss: 0.00770
# Train precision: 0.9903
# Train recall: 0.9986
# Train F1: 0.9944
# Training accuracy: 99.79 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:31<00:00,  4.81s/it]

# Validation loss: 0.33864
# Validation precision: 0.9880
# Validation recall: 0.8035
# Validation F1: 0.8863
# Validation accuracy: 96.23 (%)


# Epoch 177/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:58<00:00,  5.44s/it]

# Training loss: 0.00642
# Train precision: 0.9925
# Train recall: 0.9981
# Train F1: 0.9953
# Training accuracy: 99.82 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.83s/it]

# Validation loss: 0.35144
# Validation precision: 0.9704
# Validation recall: 0.8010
# Validation F1: 0.8776
# Validation accuracy: 95.44 (%)


# Epoch 178/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:59<00:00,  5.45s/it]

# Training loss: 0.00544
# Train precision: 0.9931
# Train recall: 0.9991
# Train F1: 0.9961
# Training accuracy: 99.85 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.83s/it]

# Validation loss: 0.38020
# Validation precision: 0.9811
# Validation recall: 0.7643
# Validation F1: 0.8592
# Validation accuracy: 95.16 (%)


# Epoch 179/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:01<00:00,  5.46s/it]

# Training loss: 0.00484
# Train precision: 0.9955
# Train recall: 0.9977
# Train F1: 0.9966
# Training accuracy: 99.87 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:33<00:00,  4.84s/it]

# Validation loss: 0.31561
# Validation precision: 0.9927
# Validation recall: 0.7938
# Validation F1: 0.8822
# Validation accuracy: 96.12 (%)


# Epoch 180/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:02<00:00,  5.47s/it]

# Training loss: 0.00572
# Train precision: 0.9936
# Train recall: 0.9986
# Train F1: 0.9961
# Training accuracy: 99.85 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.84s/it]

# Validation loss: 0.32758
# Validation precision: 0.9894
# Validation recall: 0.8125
# Validation F1: 0.8923
# Validation accuracy: 95.98 (%)


# Epoch 181/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:03<00:00,  5.47s/it]

# Training loss: 0.00480
# Train precision: 0.9949
# Train recall: 0.9977
# Train F1: 0.9963
# Training accuracy: 99.86 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.83s/it]

# Validation loss: 0.35444
# Validation precision: 0.9935
# Validation recall: 0.8234
# Validation F1: 0.9005
# Validation accuracy: 96.40 (%)


# Epoch 182/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:04<00:00,  5.48s/it]

# Training loss: 0.00466
# Train precision: 0.9944
# Train recall: 0.9991
# Train F1: 0.9968
# Training accuracy: 99.88 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.82s/it]

# Validation loss: 0.36760
# Validation precision: 0.9932
# Validation recall: 0.7956
# Validation F1: 0.8835
# Validation accuracy: 95.90 (%)


# Epoch 183/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:01<00:00,  5.47s/it]

# Training loss: 0.00542
# Train precision: 0.9946
# Train recall: 0.9982
# Train F1: 0.9964
# Training accuracy: 99.86 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:33<00:00,  4.85s/it]

# Validation loss: 0.35768
# Validation precision: 0.9739
# Validation recall: 0.8059
# Validation F1: 0.8820
# Validation accuracy: 96.08 (%)


# Epoch 184/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:55<00:00,  5.43s/it]

# Training loss: 0.01003
# Train precision: 0.9881
# Train recall: 0.9963
# Train F1: 0.9922
# Training accuracy: 99.70 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:29<00:00,  4.77s/it]

# Validation loss: 0.37524
# Validation precision: 0.9932
# Validation recall: 0.8059
# Validation F1: 0.8898
# Validation accuracy: 96.15 (%)


# Epoch 185/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:54<00:00,  5.42s/it]

# Training loss: 0.00513
# Train precision: 0.9938
# Train recall: 0.9990
# Train F1: 0.9964
# Training accuracy: 99.87 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:30<00:00,  4.80s/it]

# Validation loss: 0.34351
# Validation precision: 0.9912
# Validation recall: 0.8115
# Validation F1: 0.8924
# Validation accuracy: 96.12 (%)


# Epoch 186/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:49<00:00,  5.40s/it]

# Training loss: 0.00468
# Train precision: 0.9962
# Train recall: 0.9986
# Train F1: 0.9974
# Training accuracy: 99.90 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:29<00:00,  4.76s/it]

# Validation loss: 0.35495
# Validation precision: 0.9865
# Validation recall: 0.8066
# Validation F1: 0.8875
# Validation accuracy: 96.05 (%)


# Epoch 187/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:53<00:00,  5.42s/it]

# Training loss: 0.00562
# Train precision: 0.9939
# Train recall: 0.9986
# Train F1: 0.9962
# Training accuracy: 99.86 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:29<00:00,  4.76s/it]

# Validation loss: 0.24047
# Validation precision: 0.9201
# Validation recall: 0.8436
# Validation F1: 0.8802
# Validation accuracy: 95.87 (%)


# Epoch 188/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:49<00:00,  5.40s/it]

# Training loss: 0.00804
# Train precision: 0.9926
# Train recall: 0.9981
# Train F1: 0.9954
# Training accuracy: 99.82 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:30<00:00,  4.78s/it]

# Validation loss: 0.28953
# Validation precision: 0.9887
# Validation recall: 0.8096
# Validation F1: 0.8902
# Validation accuracy: 96.15 (%)


# Epoch 189/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [15:53<00:00,  5.42s/it]

# Training loss: 0.00330
# Train precision: 0.9968
# Train recall: 0.9986
# Train F1: 0.9977
# Training accuracy: 99.91 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.83s/it]

# Validation loss: 0.35486
# Validation precision: 0.9932
# Validation recall: 0.8022
# Validation F1: 0.8875
# Validation accuracy: 96.05 (%)


# Epoch 190/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:04<00:00,  5.48s/it]

# Training loss: 0.00609
# Train precision: 0.9936
# Train recall: 0.9986
# Train F1: 0.9961
# Training accuracy: 99.85 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:30<00:00,  4.78s/it]

# Validation loss: 0.38683
# Validation precision: 0.9902
# Validation recall: 0.7704
# Validation F1: 0.8666
# Validation accuracy: 95.55 (%)


# Epoch 191/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:05<00:00,  5.48s/it]

# Training loss: 0.00607
# Train precision: 0.9924
# Train recall: 0.9981
# Train F1: 0.9953
# Training accuracy: 99.82 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.82s/it]

# Validation loss: 0.30444
# Validation precision: 0.9862
# Validation recall: 0.7934
# Validation F1: 0.8793
# Validation accuracy: 95.80 (%)


# Epoch 192/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:00<00:00,  5.46s/it]

# Training loss: 0.00380
# Train precision: 0.9963
# Train recall: 0.9986
# Train F1: 0.9975
# Training accuracy: 99.90 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:31<00:00,  4.81s/it]

# Validation loss: 0.25493
# Validation precision: 0.9954
# Validation recall: 0.8090
# Validation F1: 0.8926
# Validation accuracy: 96.30 (%)


# Epoch 193/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:00<00:00,  5.46s/it]

# Training loss: 0.00860
# Train precision: 0.9935
# Train recall: 0.9958
# Train F1: 0.9947
# Training accuracy: 99.79 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.83s/it]

# Validation loss: 0.57496
# Validation precision: 0.9878
# Validation recall: 0.7350
# Validation F1: 0.8429
# Validation accuracy: 94.62 (%)


# Epoch 194/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:01<00:00,  5.46s/it]

# Training loss: 0.00710
# Train precision: 0.9912
# Train recall: 0.9967
# Train F1: 0.9940
# Training accuracy: 99.77 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:30<00:00,  4.79s/it]

# Validation loss: 0.36084
# Validation precision: 0.9976
# Validation recall: 0.7753
# Validation F1: 0.8725
# Validation accuracy: 95.69 (%)


# Epoch 195/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:01<00:00,  5.46s/it]

# Training loss: 0.01142
# Train precision: 0.9911
# Train recall: 0.9963
# Train F1: 0.9937
# Training accuracy: 99.76 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.82s/it]

# Validation loss: 0.51216
# Validation precision: 0.9733
# Validation recall: 0.8360
# Validation F1: 0.8994
# Validation accuracy: 96.23 (%)


# Epoch 196/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:06<00:00,  5.49s/it]

# Training loss: 0.00665
# Train precision: 0.9889
# Train recall: 0.9986
# Train F1: 0.9937
# Training accuracy: 99.76 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.82s/it]

# Validation loss: 0.38954
# Validation precision: 0.9931
# Validation recall: 0.7804
# Validation F1: 0.8740
# Validation accuracy: 95.58 (%)


# Epoch 197/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:03<00:00,  5.48s/it]

# Training loss: 0.00537
# Train precision: 0.9936
# Train recall: 0.9968
# Train F1: 0.9952
# Training accuracy: 99.81 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:30<00:00,  4.79s/it]

# Validation loss: 0.39557
# Validation precision: 0.9930
# Validation recall: 0.7723
# Validation F1: 0.8689
# Validation accuracy: 95.44 (%)


# Epoch 198/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:01<00:00,  5.47s/it]

# Training loss: 0.00840
# Train precision: 0.9935
# Train recall: 0.9967
# Train F1: 0.9951
# Training accuracy: 99.81 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:32<00:00,  4.83s/it]

# Validation loss: 0.25244
# Validation precision: 0.9250
# Validation recall: 0.8513
# Validation F1: 0.8866
# Validation accuracy: 95.62 (%)


# Epoch 199/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:05<00:00,  5.49s/it]

# Training loss: 0.00515
# Train precision: 0.9940
# Train recall: 0.9981
# Train F1: 0.9960
# Training accuracy: 99.85 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:31<00:00,  4.81s/it]

# Validation loss: 0.34744
# Validation precision: 0.9932
# Validation recall: 0.8141
# Validation F1: 0.8948
# Validation accuracy: 96.33 (%)


# Epoch 200/200

# 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [16:04<00:00,  5.48s/it]

# Training loss: 0.00870
# Train precision: 0.9922
# Train recall: 0.9977
# Train F1: 0.9949
# Training accuracy: 99.80 (%)


# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:31<00:00,  4.82s/it]

# Validation loss: 0.35956
# Validation precision: 0.8861
# Validation recall: 0.8586
# Validation F1: 0.8722
# Validation accuracy: 94.80 (%)
